# REPRODUCIBILITY PACKAGE FOR:
## Development acupuncture: The network structure of multidimensional poverty and its implications

This notebook can be used to reproduce the results of the working paper "Development acupuncture: The network structure of multidimensional poverty and its implications" by Viktor Stojkoski , Luis F. Lopez-Calva,  Kimberly Bolch , and Almudena Fernandez.

All calculations were done in Python 3.9.13 and require the following packages:

* numpy: 1.26.4

* pandas: 2.2.0

* scipy: 1.12.0

* statsmodels: 0.14.1

* matplotlib: 3.8.3

* plotly: 5.9.0

* networkx: 2.8.4

* sklearn: 1.0.2

* linearmodels: 5.4

* stargazer==0.0.6

* policy_priority_inference.py (imported from the .py file in the same directory)

In [8]:
## Import the necessary packages
import numpy as np
import pandas as pd
import os
import glob
from scipy.stats import rankdata
from scipy.stats import linregress


import statsmodels.api as sm
import statsmodels.formula.api as smf


from stargazer.stargazer import Stargazer
from statsmodels.iolib.summary2 import summary_col

from IPython.core.display import HTML
from IPython.display import HTML


import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots

import networkx as nx
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler


import policy_priority_inference as ppi



def remove_row_col(matrix, vec, var_names, variable):
    index = var_names.index(variable)
    new_matrix = np.delete(matrix, index, axis=0)  # Remove row
    new_matrix = np.delete(new_matrix, index, axis=1)  # Remove column
    new_vec = np.delete(vec, index)
    return new_matrix, vec

# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

In [9]:
TT_FINAL

,country,year,MPI,var_sample,A_int,A_avg,A_avg_raw,A_int_rank,centrality_w_norm,centrality_w_norm_rank,full_names
0,nam,7,0.381508,d_cm_01,0.005434,0.024324,0.376074,10,0.014261,10,Child mortality
1,nam,7,0.381508,d_nutr_01,0.033832,0.142377,0.347675,5,0.084066,6,Nutrition
2,nam,7,0.381508,d_satt_01,0.013156,0.064793,0.368351,8,0.038085,9,School attendance
3,nam,7,0.381508,d_educ_01,0.012470,0.066746,0.369037,9,0.039208,8,Schooling
4,nam,7,0.381508,d_elct_01,0.062034,0.273219,0.319473,3,0.160981,3,Electricity
...,...,...,...,...,...,...,...,...,...,...,...
1467,mdg,18,0.556743,d_wtr_01,0.062204,0.347358,0.494539,5,0.114861,5,Drinking water
1468,mdg,18,0.556743,d_sani_01,0.093881,0.450380,0.462862,2,0.151528,2,Sanitation
1469,mdg,18,0.556743,d_hsg_01,0.074373,0.401509,0.482370,3,0.133203,3,Housing
1470,mdg,18,0.556743,d_ckfl_01,0.099045,0.456024,0.457698,1,0.154431,1,Cooking fuel


## CLEAR DATA: ESTIMATE THE POVERTY SPACE & POVERTY CENTRALITY

The code below clears the raw data and estimates the Poverty Space and Poverty Centrality for all countries in our data sample.

In [21]:
## CLEAR DATA: ESTIMATE THE POVERTY SPACE & POVERTY CENTRALITY


var_weights = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
variables = ['d_cm_01', 'd_nutr_01', 'd_satt_01', 'd_educ_01', 'd_elct_01',
             'd_wtr_01', 'd_sani_01', 'd_hsg_01', 'd_ckfl_01', 'd_asst_01']
dimensions = [1, 1, 2, 2, 3, 3, 3, 3, 3, 3]
full_variable_names = ['Child mortality', 'Nutrition', 'School attendance', 'Schooling',
                       'Electricity', 'Drinking water', 'Sanitation', 'Housing',
                       'Cooking fuel', 'Assets']

threshold = 0

# In Python, using glob, you can search for all CSV files in the current working directory

search_pattern = os.path.join(os.getcwd(), 'data', '*.csv')
csv_files = glob.glob(search_pattern)

PHI_NORMALIZED = {}
list_names = {}
list_short_names = {}

Correlation_matrix = np.nan * np.zeros((len(csv_files), len(variables)))

                                       
columns = ['country', 'year', 'MPI', 'var_sample', 'A_int', 'A_avg', 'A_avg_raw', 'A_int_rank', 'centrality_w_norm', 'centrality_w_norm_rank', 'full_names']
TT_FINAL = pd.DataFrame(columns=columns)


for i in range(len(csv_files)):

    # Get the file name from the path
    filename = os.path.basename(csv_files[i])

    # Extract country code and year from the filename
    ctry = filename[:3]
    yyr = filename[-6:-4]  # Adjust according to your file naming convention if needed

    # Read the data
    TT = pd.read_csv(csv_files[i])

    # Create the deprivation matrix
    XX = pd.DataFrame()
    jj = []

    for j, var in enumerate(variables):
        if var in TT.columns:
            X = TT[var]
            if X.isna().mean() < 1:  # Check if the column is not entirely NaN
                XX = pd.concat([XX, X], axis=1)
            else:
                jj.append(j)  # Append j+1 to maintain MATLAB's 1-based indexing

    # Assuming previous steps have been correctly executed

    # Convert XX back to DataFrame if not already, to simplify handling NaN values
    # This step is precautionary; XX should already be a DataFrame after previous operations
    XX = pd.DataFrame(XX)

    # Filter out rows with any NaN values across all columns in XX
    # Also filter corresponding rows in the weights array
    valid_rows = ~XX.isnull().any(axis=1)
    XX_filtered = XX[valid_rows]

    weights = TT['weight'].values  # Extracting weight values from the DataFrame
    weights_sample = np.array(var_weights)  # Ensure var_weights is a numpy array for easy manipulation

    # Adjust weights_sample based on jj, ensuring jj is zero-based
    weights_sample = np.delete(weights_sample, jj)

    weights_filtered = weights[valid_rows]

    var_sample = [var for i, var in enumerate(variables) if i not in jj]
    full_names = [name for i, name in enumerate(full_variable_names) if i not in jj]


    
    # Normalize the sample weights
    weights_sample_normalized = weights_sample / sum(weights_sample)

    # Calculate the weighted deprivation matrix G
    # Note: Element-wise multiplication of weights_sample_normalized with each column in XX_filtered
    G = XX_filtered.multiply(weights_sample_normalized, axis='columns')

    # Calculate Z based on the threshold condition
    Z = np.where(G.sum(axis=1) > threshold, 1, 0) if threshold == 0 else np.where(G.sum(axis=1) >= threshold, 1, 0)

    # Calculate the final index H
    H = np.sum(weights_filtered * Z) / np.sum(weights_filtered)    


    # Ensure G is a numpy array for these operations, if not already
    G = G.to_numpy() if hasattr(G, 'to_numpy') else G

    # Recreate G_k from G
    G_k = np.copy(G)

    # Initialize xx with zeros with the correct dimensions
    # Note: Adjusting approach to ensure dimensions match expectations for numpy operations
    xx = np.zeros((np.sum(1 - Z), G_k.shape[1]))

    # Update G_k directly with numpy where applicable
    # We need to ensure we're updating rows where Z is 0 with xx, but this approach requires boolean indexing or equivalent in numpy
    not_Z_rows = np.where(Z == 0)[0]  # Get indices of rows where Z is 0

    # Since direct assignment as attempted can mismatch dimensions, we ensure alignment before attempting to replace
    # However, given xx aims to replace all not_Z_rows with zeros, we can simplify by directly setting zeros without using xx
    G_k[not_Z_rows, :] = 0  # Directly set rows where Z is 0 to zeros, aligning with intention

    # Proceed with adjusted calculations
    # Note: The calculations for W, A, A_est, and MPI should then ensure they are using numpy operations as well

    W_filtered = weights_filtered.reshape(-1, 1)  # Ensure weights_filtered is column vector for matrix operations
    A = (W_filtered * G_k) / np.sum(weights_filtered[Z == 1])

    A_est = np.sum(A, axis=0)

    MPI = np.sum(A_est * H)

    A_est *= H  # Update A_est by multiplying it with H

    # Create weighted networks
    Z_indices = np.where(Z)[0]  # Find indices where Z is True
    G_z = XX_filtered.iloc[Z_indices, :].to_numpy()  # Filter G_z based on Z

    # Ensure 'weights' and 'Z' are numpy arrays for the operations
    weights_final = np.array(weights_filtered)
    Z = np.array(Z, dtype=bool)
    A = np.array(A)  # Assuming 'A' is already correctly shaped and matches 'Z'

    # Select rows from 'A' where 'Z' is True
    A_filtered = A[Z, :]

    # Calculate the dot product of 'weights' transposed and 'Z' (which should be a scalar if both are vectors)
    weights_Z_product = np.dot(weights_final.T, Z)

    # Scale 'A_filtered' by 'weights_Z_product'
    M_w = weights_Z_product * A_filtered ## NOT CALCULATED CORRECTLY!!!!

    # Remove columns where the sum across rows in G_z is 0
    xx = np.sum(G_z > 0, axis=0)
    jj = np.where(xx == 0)[0]

    # Update var_sample and full_names by removing indices jj
    var_sample = np.delete(np.array(var_sample), jj).tolist()
    full_names = np.delete(np.array(full_names), jj).tolist()

    # Update G_z, M_w, and M_u by removing columns jj
    G_z = np.delete(G_z, jj, axis=1)
    M_w = np.delete(M_w, jj, axis=1)

    # Create similarity networks
    PHI_w_int = np.dot(M_w.T, G_z)  # Calculate initial similarity matrix
    PHI_w_norm = np.full(PHI_w_int.shape, np.nan)  # Initialize PHI_w_norm with NaN

    # Normalize PHI_w_int to create PHI_w_norm
    for k in range(PHI_w_int.shape[0]):
        for j in range(PHI_w_int.shape[1]):
            PHI_w_norm[k, j] = PHI_w_int[k, j] / PHI_w_int[j, j]

    # Adjust PHI_w_norm by subtracting the identity matrix scaled by PHI_w_norm
    PHI_w_norm = PHI_w_norm - np.eye(PHI_w_norm.shape[0]) * PHI_w_norm

    D, V = np.linalg.eig(PHI_w_norm)

    # Find the index of the eigenvalue with the maximum magnitude
    z = np.argmax(D)

    # Extract the corresponding eigenvector and compute the centrality measure
    centrality_w_norm = V[:, z]

    # Normalize the centrality measure so that its sum equals 1
    centrality_w_norm /= np.sum(centrality_w_norm)
    
    for j in range(len(variables)):
        
        try:
            index = var_sample.index(variables[j])
        except ValueError:
            
            continue
        
        Phi_reduced = np.delete(PHI_w_norm, index, axis=0).copy()  # Remove row
        Phi_reduced = np.delete(Phi_reduced, index, axis=1)  # Remove column
        centrality_reduced = np.delete(centrality_w_norm, index).copy()
        
        D, V = np.linalg.eig(Phi_reduced)

        # Find the index of the eigenvalue with the maximum magnitude
        z = np.argmax(D)

        # Extract the corresponding eigenvector and compute the centrality measure
        centrality_w_norm_reduced = V[:, z]

        # Normalize the centrality measure so that its sum equals 1
        centrality_w_norm_reduced /= np.sum(centrality_w_norm_reduced)
        centrality_reduced /= np.sum(centrality_reduced)
        
        
        # Calculate the correlation and assign it to the Correlation_matrix
        corr_matrix = np.corrcoef(centrality_w_norm_reduced, centrality_reduced)
        # corr_matrix[0, 1] or corr_matrix[1, 0] contains the correlation coefficient
        Correlation_matrix[i, j] = corr_matrix[0, 1]

  
    

    # Assuming ctry, yyr, MPI, var_sample, centrality_w_norm, A_est, PHI_w_norm, and full_names are defined

    num_vars = len(var_sample)

    # 1. Replicating values
    country = np.repeat(ctry, num_vars)
    year = np.repeat(yyr, num_vars)
    MPI_repeated = np.repeat(MPI, num_vars)

    # 2. Sorting and ranking centrality_w_norm
    centrality_w_norm_rank = rankdata(-centrality_w_norm, method='ordinal')


    # Sorting A_int and ranking
    A_int = A_est.T
    A_int = np.delete(A_int, jj)  # Remove indices as specified by jj
    A_int_rank = rankdata(-A_int, method='ordinal')


    # 3. Calculating Averages
    A_avg = PHI_w_norm @ A_int
    PHI_ones = np.ones_like(PHI_w_norm)
    PHI_ones -= np.eye(len(A_avg))
    A_avg_raw = PHI_ones @ A_int

    # 4. Creating the results table
    # Note: var_sample, centrality_w_norm, and full_names should be lists or arrays of matching length
    TT_results = pd.DataFrame({
        'country': country,
        'year': year,
        'MPI': MPI_repeated,
        'var_sample': var_sample,
        'A_int': A_int,
        'A_avg': A_avg,
        'A_avg_raw': A_avg_raw,
        'A_int_rank': A_int_rank,  # Adjust for zero-based indexing in Python
        'centrality_w_norm': centrality_w_norm,
        'centrality_w_norm_rank': centrality_w_norm_rank,
        'full_names': full_names
    })

    # Dynamically setting values in dictionaries using the current value of i
    PHI_NORMALIZED[f'phi_{ctry}_{yyr}'] = PHI_w_norm
    list_names[f'variable_names_{ctry}_{yyr}'] = full_names
    list_short_names[f'variable_names_{ctry}_{yyr}'] = var_sample


    TT_FINAL = pd.concat([TT_FINAL, TT_results], ignore_index=True)
    
print('FINISHED')

FINISHED


In [ ]:
TT_FINAL = pd.read_csv('TT_FINAL.csv', encoding='latin1')
TT_FINAL['year'] = TT_FINAL['year'].astype(str).str.zfill(2)
ctry_yr = 'Kyrgyz Republic 2006'
rows = TT_FINAL[TT_FINAL['full_country_year'] == ctry_yr].copy()
ctry_init = rows['country'].unique()
yyr_init = rows['year'].unique()


,full_country_year,country_full,year_full,country,year,MPI,var_sample,A_int,A_avg,A_avg_raw,...,centrality_w_norm,centrality_w_norm_rank,full_names,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,Namibia 2007,Namibia,2007,nam,07,0.381508,d_cm_01,0.005434,0.024324,0.376074,...,0.014261,10,Child mortality,NaN,NaN,AFG,Afghanistan,NaN,NaN,NaN
1,Namibia 2007,Namibia,2007,nam,07,0.381508,d_nutr_01,0.033832,0.142377,0.347675,...,0.084066,6,Nutrition,NaN,NaN,ALB,Albania,NaN,NaN,NaN
2,Namibia 2007,Namibia,2007,nam,07,0.381508,d_satt_01,0.013156,0.064793,0.368351,...,0.038085,9,School attendance,NaN,NaN,DZA,Algeria,NaN,NaN,NaN
3,Namibia 2007,Namibia,2007,nam,07,0.381508,d_educ_01,0.012470,0.066746,0.369037,...,0.039208,8,Schooling,NaN,NaN,ASM,American Samoa,NaN,NaN,NaN
4,Namibia 2007,Namibia,2007,nam,07,0.381508,d_elct_01,0.062034,0.273219,0.319473,...,0.160981,3,Electricity,NaN,NaN,AND,Andorra,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,Madagascar 2018,Madagascar,2018,mdg,18,0.556743,d_wtr_01,0.062204,0.347358,0.494539,...,0.114861,5,Drinking water,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1468,Madagascar 2018,Madagascar,2018,mdg,18,0.556743,d_sani_01,0.093881,0.450380,0.462862,...,0.151528,2,Sanitation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1469,Madagascar 2018,Madagascar,2018,mdg,18,0.556743,d_hsg_01,0.074373,0.401509,0.482370,...,0.133203,3,Housing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1470,Madagascar 2018,Madagascar,2018,mdg,18,0.556743,d_ckfl_01,0.099045,0.456024,0.457698,...,0.154431,1,Cooking fuel,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## GENERATE DATA FOR REGRESSION ANALYSIS

The code below generates the data used in the regression analysis (Tables 1-3).

In [12]:
## GENERATE DATA FOR REGRESSION ANALYSIS - TABLE 1

unique_pairs = TT_FINAL[['country', 'year']].drop_duplicates().reset_index(drop=True)


# Count occurrences of each unique entry in 'country'
count_occurrences = TT_FINAL['country'].value_counts()

# Initialize an empty DataFrame for regression data
TT_regression = pd.DataFrame()

# Iterate over each unique country with more than 10 occurrences
for country in count_occurrences[count_occurrences > 10].index:
    
    # Filter TT_FINAL for the current country
    TT_int = TT_FINAL[TT_FINAL['country'] == country].copy().reset_index(drop=True)
    
    # Convert 'year' to numeric and get unique years
    years = pd.to_numeric(TT_int['year'], errors='coerce')
    unique_years = years.unique()
    
    # Find the minimum and maximum years
    min_year = TT_int['year'].min()
    max_year = TT_int['year'].max()
    
    # Find all indices where 'year' is equal to the min_year and max_year
    z_initial = TT_int[TT_int['year'] == min_year].index
    z_final = TT_int[TT_int['year'] == max_year].index
    
    
    names_initial = TT_int.loc[z_initial, 'var_sample']
    names_final = TT_int.loc[z_final, 'var_sample']

    # Finding the intersection of 'var_sample' names between initial and final
    vars = set(names_initial) & set(names_final)

    # Define empty lists to store the indices
    zz_initial = []
    zz_final = []

    # Iterate over each variable name in 'vars' to find its occurrences within the initial and final years
    for var in vars:
        # Filter TT_int for rows where 'var_sample' matches the current variable name
        matching_rows = TT_int[TT_int['var_sample'] == var].index

        # Intersect matching_rows with z_initial and z_final using pandas Index intersection
        intersection_initial = matching_rows.intersection(z_initial)
        intersection_final = matching_rows.intersection(z_final)

        # Store the first index of the intersection if it exists; otherwise, store NaN
        zz_initial.append(intersection_initial[0] if not intersection_initial.empty else np.nan)
        zz_final.append(intersection_final[0] if not intersection_final.empty else np.nan)

    # Convert lists to pandas Index for further consistency in indexing if needed
    zz_initial = pd.Index(zz_initial).dropna().astype(int)
    zz_final = pd.Index(zz_final).dropna().astype(int)
            
    # Extract the initial country-year pair
    initial_pair = TT_int.iloc[z_initial][['country', 'year']]

    # Assuming 'unique_pairs' is a DataFrame with a reset index
    # Find the row in 'unique_pairs' that matches 'initial_pair'
    phi_loc = unique_pairs.reset_index().merge(initial_pair, on=['country', 'year'], how='inner')['index'].values[0]

    # Using the found index (phi_loc) to access the relevant data
    PHI_initial_key = f'phi_{country}_{min_year}'
    short_var_name_key = f'variable_names_{country}_{min_year}'

    PHI_initial = PHI_NORMALIZED.get(PHI_initial_key, None)
    short_var_names = list_short_names.get(short_var_name_key, [])

    # Filter PHI_initial based on the intersection of vars and short_var_names
    if PHI_initial is not None:
        # Find indices of vars in short_var_names
        indices = [short_var_names.index(var) for var in vars if var in short_var_names]

        # Filter PHI_initial using numpy's advanced indexing
        PHI_initial_filtered = PHI_initial[np.ix_(indices, indices)]
        
        
    # Calculate changes for each metric between initial and final observations
    change_MPI = TT_int.loc[zz_final, 'MPI'].values - TT_int.loc[zz_initial, 'MPI'].values
    change_A = TT_int.loc[zz_final, 'A_int'].values - TT_int.loc[zz_initial, 'A_int'].values
    centrality_w_norm_final = TT_int.loc[zz_final, 'centrality_w_norm'].values
    # Assuming change_A is a single value or a numpy array that matches the dimensions for multiplication with PHI_initial
    # If PHI_initial is a DataFrame, ensure it's converted to a numpy array for this operation, if not already compatible
    
    # Calculate the number of years between the initial and final observations
    # Assuming unique_yyrs is a numpy array or a pandas Series of unique years present in TT_int
    change_years = max(unique_years) - min(unique_years)

    # Normalize the changes by the number of years
    # These operations assume change_years is a scalar. If change_years is an array, adjustments might be needed
    change_A /= change_years
    
    
    # Prepare the DataFrame for regression analysis by selecting the initial observation
    TT_int_for_regression = TT_int.loc[zz_initial, :].copy()

    # Add calculated changes to the DataFrame
    TT_int_for_regression['change_MPI'] = change_MPI
    TT_int_for_regression['change_A'] = change_A
    TT_int_for_regression['change_years'] = change_years
    TT_int_for_regression['centrality_w_norm_final'] = centrality_w_norm_final

    # Append the prepared DataFrame to the TT_regression DataFrame
    TT_regression = pd.concat([TT_regression, TT_int_for_regression], ignore_index=True)
    

## ADD INSTRUMENTAL


unique_regression_pairs = TT_regression[['country', 'year']].drop_duplicates().reset_index(drop=True)

# Initialize a matrix to hold the instrumental correlations
CORRELATION_INSTRUMENTAL = np.nan * np.zeros((len(unique_regression_pairs), len(unique_regression_pairs)))

for i in range(len(unique_regression_pairs)):
    for j in range(len(unique_regression_pairs)):
        # Dynamically construct keys based on country and year
        key_i = f"phi_{unique_regression_pairs.iloc[i]['country']}_{unique_regression_pairs.iloc[i]['year']}"
        key_j = f"phi_{unique_regression_pairs.iloc[j]['country']}_{unique_regression_pairs.iloc[j]['year']}"
        names_key_i = f"variable_names_{unique_regression_pairs.iloc[i]['country']}_{unique_regression_pairs.iloc[i]['year']}"
        names_key_j = f"variable_names_{unique_regression_pairs.iloc[j]['country']}_{unique_regression_pairs.iloc[j]['year']}"

        
        PHI_i = PHI_NORMALIZED[key_i]
        names_i = list_short_names[names_key_i]

        PHI_j = PHI_NORMALIZED[key_j]
        names_j = list_short_names[names_key_j]

        # Find common variables
        vars = list(set(names_i) & set(names_j))
        
        # Indices of common variables in PHI matrices
        zz_1 = [names_i.index(var) for var in vars]
        zz_2 = [names_j.index(var) for var in vars]
        
        # Selecting the relevant rows and columns from PHI matrices based on common variables
        PHI_i_reduced = PHI_i[np.ix_(zz_1, zz_1)]
        PHI_j_reduced = PHI_j[np.ix_(zz_2, zz_2)]
        
        # Calculate correlation between flattened matrices
        corr_value = np.corrcoef(PHI_i_reduced.flatten(), PHI_j_reduced.flatten())[0, 1]
        CORRELATION_INSTRUMENTAL[i, j] = corr_value

  #  print(f"Processed pair {i+1} of {len(unique_regression_pairs)}")
        
# Adjust the correlation matrix
CORRELATION_INSTRUMENTAL -= np.eye(len(unique_regression_pairs))

# Find max correlation indices
max_indices = np.argmax(CORRELATION_INSTRUMENTAL, axis=1)

# Initialize centrality_w_norm_inst as NaNs
centrality_w_norm_inst = np.nan * np.ones(len(TT_regression['centrality_w_norm']))

for i in range(len(unique_regression_pairs)):
    country_i, year_i = unique_regression_pairs.iloc[i]['country'], unique_regression_pairs.iloc[i]['year']
    country_j, year_j = unique_regression_pairs.iloc[max_indices[i]]['country'], unique_regression_pairs.iloc[max_indices[i]]['year']

    # Filter TT_regression_2 for the current and max correlated pairs
    pair_i = TT_regression[(TT_regression['country'] == country_i) & (TT_regression['year'] == year_i)]
    pair_j = TT_regression[(TT_regression['country'] == country_j) & (TT_regression['year'] == year_j)]

    # Common variables
    vars = list(set(pair_i['var_sample']) & set(pair_j['var_sample']))

    # Find indices in TT_regression_2 for vars in both pairs
    for var in vars:
        index_i = pair_i[pair_i['var_sample'] == var].index
        index_j = pair_j[pair_j['var_sample'] == var].index

        # Update centrality_w_norm_inst based on common variables
        centrality_w_norm_inst[index_i] = pair_j.loc[index_j, 'centrality_w_norm'].values
    
TT_regression['centrality_w_norm_inst'] = centrality_w_norm_inst

## GENERATE DATA FOR 3 PERIOD REGRESSION

# Get unique pairs from the first two columns
unique_pairs = TT_FINAL.iloc[:, 0:2].drop_duplicates()

# Convert country column to string type, find unique entries and their counts
unique_entries, entry_counts = np.unique(TT_FINAL['country'].astype(str), return_counts=True)
count_occurrences = np.bincount(entry_counts)

TT_regression_3_period = pd.DataFrame()
for i, unique_entry in enumerate(unique_entries):
    if (unique_pairs['country'] == unique_entry).sum() == 3:
        indices = TT_FINAL[TT_FINAL['country'] == unique_entry].index
        TT_int = TT_FINAL.loc[indices]

        # Convert 'year' to numeric and get unique years
        years = pd.to_numeric(TT_int['year'], errors='coerce')
        unique_years = np.sort(years.unique())
    

        for KK in range(len(unique_years) - 1):
            indices_initial = TT_int[years == unique_years[KK]].index
            indices_final = TT_int[years == unique_years[KK + 1]].index

            names_initial = TT_int.loc[indices_initial, 'var_sample']
            names_final = TT_int.loc[indices_final, 'var_sample']

            vars = np.intersect1d(names_initial, names_final)
            zz_initial = np.full(vars.shape, np.nan)
            zz_final = np.full(vars.shape, np.nan)

            for j, var in enumerate(vars):
                z = TT_int[TT_int['var_sample'] == var].index
                zz_initial[j] = np.intersect1d(indices_initial, z)
                zz_final[j] = np.intersect1d(indices_final, z)

            initial_pair = TT_int.loc[indices_initial, :].iloc[:, 0:2].drop_duplicates()
            
             # Using the found index (phi_loc) to access the relevant data
            PHI_initial_key = f'phi_{country}_{unique_years[KK]}'
            short_var_name_key = f'variable_names_{country}_{unique_years[KK]}'

            PHI_initial = PHI_NORMALIZED.get(PHI_initial_key, None)
            short_var_names = list_short_names.get(short_var_name_key, [])

            # Filter PHI_initial based on the intersection of vars and short_var_names
            if PHI_initial is not None:
                # Find indices of vars in short_var_names
                indices = [short_var_names.index(var) for var in vars if var in short_var_names]

                # Filter PHI_initial using numpy's advanced indexing
                PHI_initial_filtered = PHI_initial[np.ix_(indices, indices)]
                
                    # Calculate changes for each metric between initial and final observations
            change_MPI = TT_int.loc[zz_final, 'MPI'].values - TT_int.loc[zz_initial, 'MPI'].values
            change_A = TT_int.loc[zz_final, 'A_int'].values - TT_int.loc[zz_initial, 'A_int'].values


            # Calculate the number of years between the initial and final observations
            # Assuming unique_yyrs is a numpy array or a pandas Series of unique years present in TT_int
            change_years = unique_years[KK+1] - unique_years[KK]
            

            # Normalize the changes by the number of years
            # These operations assume change_years is a scalar. If change_years is an array, adjustments might be needed
            change_A /= change_years


            # Prepare the DataFrame for regression analysis by selecting the initial observation
            TT_int_for_regression = TT_int.loc[zz_initial, :].copy()

            # Add calculated changes to the DataFrame
            TT_int_for_regression['change_MPI'] = change_MPI
            TT_int_for_regression['change_A'] = change_A
            TT_int_for_regression['change_years'] = change_years
            TT_int_for_regression['period'] = KK            

            # Append the prepared DataFrame to the TT_regression DataFrame
            TT_regression_3_period = pd.concat([TT_regression_3_period, TT_int_for_regression], ignore_index=True)
            
            
print('FINISHED')            

FINISHED


In [ ]:
PHI_NORMALIZED

,country,year,MPI,var_sample,A_int,A_avg,A_avg_raw,A_int_rank,centrality_w_norm,centrality_w_norm_rank,full_names
0,nam,07,0.381508,d_cm_01,0.005434,0.024324,0.376074,10,0.014261,10,Child mortality
1,nam,07,0.381508,d_nutr_01,0.033832,0.142377,0.347675,5,0.084066,6,Nutrition
2,nam,07,0.381508,d_satt_01,0.013156,0.064793,0.368351,8,0.038085,9,School attendance
3,nam,07,0.381508,d_educ_01,0.012470,0.066746,0.369037,9,0.039208,8,Schooling
4,nam,07,0.381508,d_elct_01,0.062034,0.273219,0.319473,3,0.160981,3,Electricity
...,...,...,...,...,...,...,...,...,...,...,...
1467,mdg,18,0.556743,d_wtr_01,0.062204,0.347358,0.494539,5,0.114861,5,Drinking water
1468,mdg,18,0.556743,d_sani_01,0.093881,0.450380,0.462862,2,0.151528,2,Sanitation
1469,mdg,18,0.556743,d_hsg_01,0.074373,0.401509,0.482370,3,0.133203,3,Housing
1470,mdg,18,0.556743,d_ckfl_01,0.099045,0.456024,0.457698,1,0.154431,1,Cooking fuel


## FIGURE 1 A

The code below reproduces Figure 1 a (the Poverty Space of Turkmenistan in 2006 and 2019). In these networks, nodes are poverty indicators, and edges between them indicate significant connections. The node size corresponds to its poverty centrality, while its color matches the censored headcount ratio of the indicator.

We see that for Turkmenistan, Housing was the most central poverty indicator in 2006 (located in the core), while Electricity was the least central (located in the periphery). This suggests a household in 2006 Turkmenistan, if deprived in any other indicator, was most likely to also lack adequate Housing. Conversely, if a household was deprived in any indicator, it was the least probable to also be deprived in Electricity. Various factors such as geographic location, access to quality services, systemic inequality, and socio-economic policies could contribute to this specific poverty structure. The Poverty Space compiles these factors into a network visualization, revealing the underlying relationships between poverty indicators. By 2019, Child Mortality became the most central indicator in Turkmenistan, signifying dynamic relationships among poverty indicators that adapt in response to socio-economic changes and potential policy interventions.

In [ ]:
## PLOT FIGURE 1 A

# Kyrgyzstan in 2006
ctry_init = 'kgz'
yyr_init = '06'


PHI_w_norm = PHI_NORMALIZED[f'phi_{ctry_init}_{yyr_init}'].copy()
filtered_df = TT_FINAL[(TT_FINAL['year'] == yyr_init) & (TT_FINAL['country'] == ctry_init)]
A_int = filtered_df[['A_int']].copy()
A_int_flat = filtered_df['A_int'].copy()
var_sample = filtered_df[['full_names']].copy()
centrality_w_norm = filtered_df[['centrality_w_norm']].copy()
centrality_w_norm_flat = filtered_df['centrality_w_norm'].copy()


# Normalize and scale A_int values for coloring
pos_colors = zscore(A_int.squeeze().to_numpy())
scaler = MinMaxScaler(feature_range=(1, 100))
pos_colors_scaled = scaler.fit_transform(pos_colors.reshape(-1, 1)).flatten().round().astype(int)

# Use a matplotlib colormap similar to MATLAB's parula
cmap = plt.cm.get_cmap('viridis', 100)
node_colors = np.array([cmap(pos_colors_scaled[k]-1) for k in range(len(pos_colors_scaled))])

# Convert to Plotly colorscale
plotly_cmap = [[i/99, f'rgb({c[0]*255},{c[1]*255},{c[2]*255})'] for i, c in enumerate(cmap.colors)]

# Node labels
node_labels = var_sample['full_names'].tolist()
node_labels_dict = {i: label for i, label in enumerate(node_labels)}

# Graph threshold and construction
thresh = np.min(np.max(PHI_w_norm, axis=1))
XX = np.copy(PHI_w_norm)
XX[XX < thresh] = 0
XX = np.round(XX / np.min(XX[XX > 0]))
XX = XX + XX.T
thresh = np.min(np.max(XX, axis=1))
XX = np.copy(XX)
XX[XX < thresh] = 0
XX = (XX > 0).astype(float)
G = nx.from_numpy_matrix(XX)

                # Assuming A_int_flat and centrality_w_norm_flat are pandas Series or numpy arrays
hover_texts = []
for i in range(len(G.nodes())):
    label = node_labels_dict[i]
    CHR_value = A_int_flat.iloc[i] if isinstance(A_int_flat, pd.Series) else A_int_flat[i]
    PC_value = centrality_w_norm_flat.iloc[i] if isinstance(centrality_w_norm_flat, pd.Series) else centrality_w_norm_flat[i]
    hover_text = f"{label}<br>CHR = {CHR_value:.2f}<br>PC = {PC_value:.2f}"
    hover_texts.append(hover_text)

# Fix random state for layout
pos = nx.spring_layout(G, seed=42)

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines',
    showlegend=False)

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

# Scale node sizes based on centrality
scaler = MinMaxScaler(feature_range=(10, 50))  # Adjust the range as needed
centrality_scaled = scaler.fit_transform(centrality_w_norm_flat.values.reshape(-1, 1)).flatten()

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    textposition="top center",
    text=[node_labels_dict[i] for i in range(len(G.nodes()))],
    hovertext=hover_texts,
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale=plotly_cmap,
        size=centrality_scaled,  # Use the scaled centrality for node size
        color=pos_colors_scaled,
        colorbar=dict(
            thickness=15,
            title='Poverty Intensity',
            xanchor='center',
            titleside='top',
            tickvals=[1, 100],
            ticktext=['Low', 'High'],
            orientation='h',
            x=0.5,
            y=-0.1
        ),
        line_width=2),
    showlegend=False)

# Create dummy traces for node size legend
size_legend = [dict(size=10, label='Low Centrality'),
               dict(size=30, label='Medium Centrality'),
               dict(size=50, label='High Centrality')]

legend_traces = []
for item in size_legend:
    legend_traces.append(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(
            size=item['size'],
            color='lightgray',
            line=dict(width=1, color='black')
        ),
        legendgroup='Node Size',
        showlegend=True,
        name=item['label']
    ))

# Create figure
fig = go.Figure(data=[edge_trace, node_trace] + legend_traces)

# Update layout
fig.update_layout(
    height=500,
    width=800,
    title=f'{ctry_init.upper()} {yyr_init}',
    titlefont_size=16,
    showlegend=True,
    hovermode='closest',
    paper_bgcolor='white',  # Set background color to white
    plot_bgcolor='white',  # Set plot area background color to white
    margin=dict(b=80, l=40, r=40, t=40),  # Increase bottom margin to fit the colorbar
    annotations=[dict(
        text="Poverty Intensity",
        showarrow=False,
        xref="paper", yref="paper",
        x=0.5,
        y=-0.1,
        xanchor='center',
        yanchor='bottom'
    )],
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),  # Hide x-axis ticks
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)  # Hide y-axis ticks
)

fig.show()

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore
import networkx as nx
import os
import plotly.io as pio
import re  # Import regular expressions


for key in PHI_NORMALIZED.keys():
    try:
        # Extract country and year from the key
        match = re.match(r'phi_([a-z]+)_([0-9]+)', key)
        if not match:
            continue  # Skip if the key format is unexpected
        ctry_init, yyr_init = match.groups()
                
        PHI_w_norm = PHI_NORMALIZED[f'phi_{ctry_init}_{yyr_init}'].copy()
        filtered_df = TT_FINAL[(TT_FINAL['year'] == yyr_init) & (TT_FINAL['country'] == ctry_init)]
        A_int = filtered_df[['A_int']].copy()
        A_int_flat = filtered_df['A_int'].copy()
        var_sample = filtered_df[['full_names']].copy()
        centrality_w_norm = filtered_df[['centrality_w_norm']].copy()
        centrality_w_norm_flat = filtered_df['centrality_w_norm'].copy()

        # Normalize and scale A_int values for coloring
        pos_colors = zscore(A_int.squeeze().to_numpy())
        scaler = MinMaxScaler(feature_range=(1, 100))
        pos_colors_scaled = scaler.fit_transform(pos_colors.reshape(-1, 1)).flatten().round().astype(int)

        # Use a matplotlib colormap similar to MATLAB's parula
        cmap = plt.cm.get_cmap('viridis', 100)
        node_colors = np.array([cmap(pos_colors_scaled[k]-1) for k in range(len(pos_colors_scaled))])

        # Convert to Plotly colorscale
        plotly_cmap = [[i/99, f'rgb({c[0]*255},{c[1]*255},{c[2]*255})'] for i, c in enumerate(cmap.colors)]

        # Node labels
        node_labels = var_sample['full_names'].tolist()
        node_labels_dict = {i: label for i, label in enumerate(node_labels)}

        # Graph threshold and construction
        thresh = np.min(np.max(PHI_w_norm, axis=1))
        XX = np.copy(PHI_w_norm)
        XX[XX < thresh] = 0
        XX = np.round(XX / np.min(XX[XX > 0]))
        XX = XX + XX.T
        thresh = np.min(np.max(XX, axis=1))
        XX = np.copy(XX)
        XX[XX < thresh] = 0
        XX = (XX > 0).astype(float)
        G = nx.from_numpy_matrix(XX)
        
        
                # Assuming A_int_flat and centrality_w_norm_flat are pandas Series or numpy arrays
        hover_texts = []
        for i in range(len(G.nodes())):
            label = node_labels_dict[i]
            CHR_value = A_int_flat.iloc[i] if isinstance(A_int_flat, pd.Series) else A_int_flat[i]
            PC_value = centrality_w_norm_flat.iloc[i] if isinstance(centrality_w_norm_flat, pd.Series) else centrality_w_norm_flat[i]
            hover_text = f"{label}<br>CHR = {CHR_value:.2f}<br>PC = {PC_value:.2f}"
            hover_texts.append(hover_text)

        # Fix random state for layout
        pos = nx.spring_layout(G, seed=42)

        edge_x = []
        edge_y = []
        for edge in G.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_x.append(x0)
            edge_x.append(x1)
            edge_x.append(None)
            edge_y.append(y0)
            edge_y.append(y1)
            edge_y.append(None)

        edge_trace = go.Scatter(
            x=edge_x, y=edge_y,
            line=dict(width=0.5, color='#888'),
            hoverinfo='none',
            mode='lines',
            showlegend=False)

        node_x = []
        node_y = []
        for node in G.nodes():
            x, y = pos[node]
            node_x.append(x)
            node_y.append(y)

        # Scale node sizes based on centrality
        scaler = MinMaxScaler(feature_range=(10, 50))  # Adjust the range as needed
        centrality_scaled = scaler.fit_transform(centrality_w_norm_flat.values.reshape(-1, 1)).flatten()

        node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        textposition="top center",
        text=[node_labels_dict[i] for i in range(len(G.nodes()))],
        hovertext=hover_texts,
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale=plotly_cmap,
            size=centrality_scaled,  # Use the scaled centrality for node size
            color=pos_colors_scaled,
            colorbar=dict(
                thickness=15,
                title='Poverty Intensity',
                xanchor='center',
                titleside='top',
                tickvals=[1, 100],
                ticktext=['Low', 'High'],
                orientation='h',
                x=0.5,
                y=-0.1
            ),
            line_width=2),
        showlegend=False)

        # Create dummy traces for node size legend
        size_legend = [dict(size=10, label='Low Centrality'),
                       dict(size=30, label='Medium Centrality'),
                       dict(size=50, label='High Centrality')]

        legend_traces = []
        for item in size_legend:
            legend_traces.append(go.Scatter(
                x=[None], y=[None],
                mode='markers',
                marker=dict(
                    size=item['size'],
                    color='lightgray',
                    line=dict(width=1, color='black')
                ),
                legendgroup='Node Size',
                showlegend=True,
                name=item['label']
            ))

        # Create figure
        fig = go.Figure(data=[edge_trace, node_trace] + legend_traces)

        # Update layout
        fig.update_layout(
            height=500,
            width=800,
            title=f'{ctry_init.upper()} {yyr_init}',
            titlefont_size=16,
            showlegend=True,
            hovermode='closest',
            paper_bgcolor='white',  # Set background color to white
            plot_bgcolor='white',  # Set plot area background color to white
            margin=dict(b=80, l=40, r=40, t=40),  # Increase bottom margin to fit the colorbar
            annotations=[dict(
                text="Poverty Intensity",
                showarrow=False,
                xref="paper", yref="paper",
                x=0.5,
                y=-0.1,
                xanchor='center',
                yanchor='bottom'
            )],
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),  # Hide x-axis ticks
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)  # Hide y-axis ticks
        )



        # Save the figure as HTML
        html_filename = f'output/figure_{ctry_init}_{yyr_init}.html'
        pio.write_html(fig, file=html_filename, full_html=False, include_plotlyjs='cdn')

        # Save matrices as CSV
        PHI_w_norm_df = pd.DataFrame(PHI_w_norm, index=node_labels, columns=node_labels)
        PHI_w_norm_df.to_csv(f'output/Network_data_weighted_{ctry_init}_{yyr_init}.csv')
        XX_df = pd.DataFrame(XX, index=node_labels, columns=node_labels)
        XX_df.to_csv(f'output/Network_data_binary_{ctry_init}_{yyr_init}.csv')

    except Exception as e:
        print(f"Error processing {ctry_init} {yyr_init}: {e}")
        

In [ ]:
pip install dash plotly pandas numpy scikit-learn networkx


In [ ]:
import os

# Directory where the HTML files are stored
output_dir = 'output'
embed_dir = 'embed_codes'
os.makedirs(embed_dir, exist_ok=True)

# Loop through each HTML file in the output directory
for filename in os.listdir(output_dir):
    if filename.endswith(".html"):
        file_path = os.path.join(output_dir, filename)
        
        # Read the content of the HTML file
        with open(file_path, 'r') as f:
            html_content = f.read()
        
        # Find the start and end of the <div> and <script> tags for the plot
        div_start = html_content.find('<div')
        script_start = html_content.find('<script')
        script_end = html_content.rfind('</script>') + len('</script>')

        # Extract the <div> and <script> sections as the embed code
        embed_code = html_content[div_start:script_end]

        # Save the embed code to a new text file
        embed_filename = os.path.join(embed_dir, filename.replace(".html", "_embed_code.txt"))
        with open(embed_filename, 'w') as embed_file:
            embed_file.write(embed_code)

        print(f"Embed code for {filename} saved to {embed_filename}")

In [ ]:
TT_FINAL.to_excel('tables-data.xlsx')

## FIGURE 1 B

The code below reproduces Figure 1 a (the Poverty Space of Ethiopia in 2011 and 2019). In these networks, nodes are poverty indicators, and edges between them indicate significant connections. The node size corresponds to its poverty centrality, while its color matches the censored headcount ratio of the indicator.

In Ethiopia, the Poverty Space structure remained largely consistent over time. In both 2011 and 2019, Cooking Fuel was the most central indicator (located in the core), while Child Mortality was the least central (located in the periphery).

In [ ]:
## PLOT FIGURE 1 B


# ETHIOPIA in 2011
ctry_init = 'eth'
yyr_init = '11'


PHI_w_norm = PHI_NORMALIZED[f'phi_{ctry_init}_{yyr_init}'].copy()
filtered_df = TT_FINAL[(TT_FINAL['year'] == yyr_init) & (TT_FINAL['country'] == ctry_init)]
A_int = filtered_df[['A_int']].copy()
var_sample = filtered_df[['full_names']].copy()
centrality_w_norm = filtered_df[['centrality_w_norm']].copy()

# Normalize and scale A_int values for coloring
pos_colors = zscore(A_int.squeeze().to_numpy())
scaler = MinMaxScaler(feature_range=(1, 100))
pos_colors_scaled = scaler.fit_transform(pos_colors.reshape(-1, 1)).flatten().round().astype(int)

# Use a matplotlib colormap similar to MATLAB's parula
cmap = plt.cm.get_cmap('viridis', 100)
node_colors = np.array([cmap(pos_colors_scaled[k]-1) for k in range(len(pos_colors_scaled))])

# Convert to Plotly colorscale
plotly_cmap = [[i/99, f'rgb({c[0]*255},{c[1]*255},{c[2]*255})'] for i, c in enumerate(cmap.colors)]

# Node labels
node_labels = var_sample['full_names'].tolist()
node_labels_dict = {i: label for i, label in enumerate(node_labels)}

# Graph threshold and construction
thresh = np.min(np.max(PHI_w_norm, axis=1))
XX = np.copy(PHI_w_norm)
XX[XX < thresh] = 0
XX = np.round(XX / np.min(XX[XX > 0]))
XX = XX + XX.T
thresh = np.min(np.max(XX, axis=1))
XX = np.copy(XX)
XX[XX < thresh] = 0
XX = (XX > 0).astype(float)
G = nx.from_numpy_matrix(XX)

# Sort var_sample based on centrality_w_norm for ranking
centrality_w_norm_flat = centrality_w_norm.squeeze()
A_int_flat = A_int.squeeze()
ranked_df = var_sample.assign(A_int=A_int_flat)
ranked_df = ranked_df.assign(centrality=centrality_w_norm_flat).sort_values(by='centrality', ascending=False)
ranked_df.reset_index(drop=True, inplace=True)
ranked_df['Rank'] = ranked_df.index + 1
table_df = ranked_df[['Rank', 'full_names','A_int','centrality']]
table_df.columns = ['Rank', 'Name','CHR','PC']

# Fix random state for layout
pos = nx.spring_layout(G, seed=42)

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

# Scale node sizes based on centrality
scaler = MinMaxScaler(feature_range=(10, 50))  # Adjust the range as needed
centrality_scaled = scaler.fit_transform(centrality_w_norm_flat.values.reshape(-1, 1)).flatten()

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    textposition="top center",
    text=[node_labels_dict[i] for i in range(len(G.nodes()))],
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale=plotly_cmap,
        size=centrality_scaled,  # Use the scaled centrality for node size
        color=pos_colors_scaled,
        colorbar=dict(
            thickness=15,
            title='Poverty Intensity',
            xanchor='center',
            titleside='top',
            tickvals=[1, 100],
            ticktext=['Low', 'High'],
            orientation='h',
            x=0.5,
            y=-0.1
        ),
        line_width=2))

# Adjusting layout to separate the graph and the table
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.75, 0.25],  # Set the relative widths of the columns
    specs=[[{"type": "scatter"}, {"type": "table"}]]
)

# Add network graph
fig.add_trace(edge_trace, row=1, col=1)
fig.add_trace(node_trace, row=1, col=1)

# Add table
table_trace = go.Table(
    header=dict(
        values=['PC Rank', 'Indicator','CHR','PC'],
        fill_color='paleturquoise',
        align='left'
    ),
    cells=dict(
        values=[table_df.Rank, table_df.Name,np.round(table_df.CHR,4),np.round(table_df.PC,4)],
        fill_color='lavender',
        align='left'
    ))

fig.add_trace(table_trace, row=1, col=2)

# Update layout to separate graph and table, with white background and no x/y axis ticks
fig.update_layout(
    height=500,
    width=1000,
    title='ETHIOPIA 2011',
    titlefont_size=16,
    showlegend=False,
    hovermode='closest',
    paper_bgcolor='white',  # Set background color to white
    plot_bgcolor='white',  # Set plot area background color to white
    margin=dict(b=80, l=40, r=40, t=40),  # Increase bottom margin to fit the colorbar
    annotations=[dict(
        text="Poverty Intensity",
        showarrow=False,
        xref="paper", yref="paper",
        x=0.5,
        y=-0.1,
        xanchor='center',
        yanchor='bottom'
    )],
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),  # Hide x-axis ticks
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)  # Hide y-axis ticks
)

fig.show()
pio.write_image(fig, 'output/figure1b_eth11.pdf', format='pdf', width=1200, height=800, scale=2)

# ETHIOPIA in 2019
ctry_init = 'eth'
yyr_init = '19'

PHI_w_norm = PHI_NORMALIZED[f'phi_{ctry_init}_{yyr_init}'].copy()
filtered_df = TT_FINAL[(TT_FINAL['year'] == yyr_init) & (TT_FINAL['country'] == ctry_init)]
A_int = filtered_df[['A_int']].copy()
var_sample = filtered_df[['full_names']].copy()
centrality_w_norm = filtered_df[['centrality_w_norm']].copy()

# Normalize and scale A_int values for coloring
pos_colors = zscore(A_int.squeeze().to_numpy())
scaler = MinMaxScaler(feature_range=(1, 100))
pos_colors_scaled = scaler.fit_transform(pos_colors.reshape(-1, 1)).flatten().round().astype(int)

# Use a matplotlib colormap similar to MATLAB's parula
cmap = plt.cm.get_cmap('viridis', 100)
node_colors = np.array([cmap(pos_colors_scaled[k]-1) for k in range(len(pos_colors_scaled))])

# Convert to Plotly colorscale
plotly_cmap = [[i/99, f'rgb({c[0]*255},{c[1]*255},{c[2]*255})'] for i, c in enumerate(cmap.colors)]

# Node labels
node_labels = var_sample['full_names'].tolist()
node_labels_dict = {i: label for i, label in enumerate(node_labels)}

# Graph threshold and construction
thresh = np.min(np.max(PHI_w_norm, axis=1))
XX = np.copy(PHI_w_norm)
XX[XX < thresh] = 0
XX = np.round(XX / np.min(XX[XX > 0]))
XX = XX + XX.T
thresh = np.min(np.max(XX, axis=1))
XX = np.copy(XX)
XX[XX < thresh] = 0
XX = (XX > 0).astype(float)
G = nx.from_numpy_matrix(XX)

# Sort var_sample based on centrality_w_norm for ranking
centrality_w_norm_flat = centrality_w_norm.squeeze()
A_int_flat = A_int.squeeze()
ranked_df = var_sample.assign(A_int=A_int_flat)
ranked_df = ranked_df.assign(centrality=centrality_w_norm_flat).sort_values(by='centrality', ascending=False)
ranked_df.reset_index(drop=True, inplace=True)
ranked_df['Rank'] = ranked_df.index + 1
table_df = ranked_df[['Rank', 'full_names','A_int','centrality']]
table_df.columns = ['Rank', 'Name','CHR','PC']

# Fix random state for layout
pos = nx.spring_layout(G, seed=42)

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

# Scale node sizes based on centrality
scaler = MinMaxScaler(feature_range=(10, 50))  # Adjust the range as needed
centrality_scaled = scaler.fit_transform(centrality_w_norm_flat.values.reshape(-1, 1)).flatten()

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    textposition="top center",
    text=[node_labels_dict[i] for i in range(len(G.nodes()))],
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale=plotly_cmap,
        size=centrality_scaled,  # Use the scaled centrality for node size
        color=pos_colors_scaled,
        colorbar=dict(
            thickness=15,
            title='Poverty Intensity',
            xanchor='center',
            titleside='top',
            tickvals=[1, 100],
            ticktext=['Low', 'High'],
            orientation='h',
            x=0.5,
            y=-0.1
        ),
        line_width=2))

# Adjusting layout to separate the graph and the table
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.75, 0.25],  # Set the relative widths of the columns
    specs=[[{"type": "scatter"}, {"type": "table"}]]
)

# Add network graph
fig.add_trace(edge_trace, row=1, col=1)
fig.add_trace(node_trace, row=1, col=1)

# Add table
table_trace = go.Table(
    header=dict(
        values=['PC Rank', 'Indicator','CHR','PC'],
        fill_color='paleturquoise',
        align='left'
    ),
    cells=dict(
        values=[table_df.Rank, table_df.Name,np.round(table_df.CHR,4),np.round(table_df.PC,4)],
        fill_color='lavender',
        align='left'
    ))

fig.add_trace(table_trace, row=1, col=2)

# Update layout to separate graph and table, with white background and no x/y axis ticks
fig.update_layout(
    height=500,
    width=1000,
    title='ETHIOPIA 2019',
    titlefont_size=16,
    showlegend=False,
    hovermode='closest',
    paper_bgcolor='white',  # Set background color to white
    plot_bgcolor='white',  # Set plot area background color to white
    margin=dict(b=80, l=40, r=40, t=40),  # Increase bottom margin to fit the colorbar
    annotations=[dict(
        text="Poverty Intensity",
        showarrow=False,
        xref="paper", yref="paper",
        x=0.5,
        y=-0.1,
        xanchor='center',
        yanchor='bottom'
    )],
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),  # Hide x-axis ticks
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)  # Hide y-axis ticks
)

fig.show()
pio.write_image(fig, 'output/figure1b_eth19.pdf', format='pdf', width=1200, height=800, scale=2)


## FIGURE 2 A

In Figure 2 a, we investigate the relationship between the poverty space and the censored headcount ratio, by pooling data from every country and every period. We find a correlation between poverty centrality and the headcount ratio . Though, this correlation is only moderate, suggesting that the poverty space and the headcount ratio do not offer the same information about the structure of multidimensional poverty.

## FIGURE 2 B

In Figure 2 b, we determined the median centrality for each indicator across all countries, using only data from the final survey year. This measurement provides a snapshot of an indicator's "typical" importance across countries, offering a benchmark to assess the uniqueness of a country's poverty structure. On average, Cooking Fuel emerges as the most central indicator (located in the core), with Child Mortality being the least central (located in the periphery). 

## FIGURE 2 C

We evaluated the sensitivity of the Poverty Space in each country to missing poverty indicators by removing one poverty indicator at a time, recalculating the structural dependence matrix, and comparing the Poverty Centrality of the rest of the indicators in the recalculated matrix to their original values. Figure 2 c displays boxplots representing the distribution of the correlation between the poverty centrality measures when one indicator is removed. Our analysis reveals that this correlation is almost always above 0.9. This lack of significant differences suggests the robustness of the Poverty Space.

## FIGURE 2 D

Bar chart for each country giving the correlations between the centrality of an indicator in the initial and final survey year.

In [ ]:
# Figure 2A
correlation_coef = np.corrcoef(TT_FINAL['centrality_w_norm'], TT_FINAL['A_int'])[0, 1]
slope, intercept, r_value, p_value, std_err = linregress(TT_FINAL['centrality_w_norm'], TT_FINAL['A_int'])
regression_line = slope * TT_FINAL['centrality_w_norm'] + intercept

fig = make_subplots(rows=2, cols=3, subplot_titles=('a', 'b', 'c', 'd'), horizontal_spacing=0.08, vertical_spacing=0.15,
                    specs=[[{"type": "scatter"}, {"type": "bar"}, {"type": "box"}],
                           [{"colspan": 3}, None, None]])

# Scatter plot
fig.add_trace(go.Scatter(
    x=TT_FINAL['centrality_w_norm'],
    y=TT_FINAL['A_int'],
    mode='markers',
    marker=dict(color='rgba(31, 119, 180, 0.8)', size=8, line=dict(color='DarkSlateGrey', width=1)),
    name='Data Points'
), row=1, col=1)

# Regression line
fig.add_trace(go.Scatter(
    x=TT_FINAL['centrality_w_norm'],
    y=regression_line,
    mode='lines',
    line=dict(color='rgba(255, 0, 0, 0.8)', width=2, dash='dash'),
    name='Regression Line'
), row=1, col=1)

fig.update_xaxes(title_text='Poverty Centrality', showline=True, linewidth=2, linecolor='black', row=1, col=1,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))
fig.update_yaxes(title_text='Censored Headcount Ratio', showline=True, linewidth=2, linecolor='black', row=1, col=1,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))

fig.add_annotation(
    x=0.01, y=0.98, xref='paper', yref='paper',
    text=f'Correlation Coefficient: {correlation_coef:.2f}',
    showarrow=False,
    font=dict(size=12, color='black'),
    bgcolor='white',
    opacity=1
)

# Figure 2B
median_indicator = TT_regression.groupby('full_names')['centrality_w_norm_final'].median().reset_index()
median_indicator_sorted_general = median_indicator.sort_values(by='centrality_w_norm_final')

# Horizontal bar plot
fig.add_trace(go.Bar(
    x=median_indicator_sorted_general['centrality_w_norm_final'],
    y=median_indicator_sorted_general['full_names'],
    orientation='h',
    marker=dict(color='rgba(31, 119, 180, 0.8)', line=dict(color='DarkSlateGrey', width=1))
), row=1, col=2)

fig.update_xaxes(title_text='Median Poverty Centrality', showline=True, linewidth=2, linecolor='black', row=1, col=2,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True, row=1, col=2,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))

# Figure 2C
sorted_full_variable_names = median_indicator_sorted_general['full_names'].tolist()
sorted_indices = [full_variable_names.index(name) for name in sorted_full_variable_names]

for i in range(len(sorted_indices)):
    column_data = Correlation_matrix[:, sorted_indices[i]]
    fig.add_trace(go.Box(
        x=column_data[~np.isnan(column_data)],
        name=sorted_full_variable_names[i],
        orientation='h',
        boxmean='sd',
        marker_color='rgba(31, 119, 180, 0.8)'
    ), row=1, col=3)

fig.update_layout(
    width=1200,
    height=1000,
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=False
)

fig.update_xaxes(title_text='Correlation Coefficient', showline=True, linewidth=2, linecolor='black', row=1, col=3,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))
fig.update_yaxes(showticklabels=False, row=1, col=3)  # Hide y-axis labels for 2C

# Align y-axis labels for 2B and 2C
fig.update_yaxes(tickmode='array', tickvals=list(range(len(sorted_full_variable_names))), ticktext=sorted_full_variable_names, row=1, col=2,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))

# Figure 2D
unique_countries = TT_regression['country'].unique()

# Initialize a list to store correlation values for each country
corr_indicators = []

# Calculate the correlation for 'centrality_w_norm' and 'centrality_w_norm_final' for each country
for country in unique_countries:
    # Filter the DataFrame for the current country
    filtered_df = TT_regression[TT_regression['country'] == country]
    
    # Calculate correlation and append to list
    correlation = filtered_df['centrality_w_norm'].corr(filtered_df['centrality_w_norm_final'])
    corr_indicators.append(correlation)

# Convert the list of correlations to a NumPy array for sorting
corr_indicators = np.array(corr_indicators)

# Sort the correlations and get the sorted indices
sorted_indices = np.argsort(corr_indicators)

# Vertical bar plot for correlations
fig.add_trace(go.Bar(
    x=np.array(unique_countries)[sorted_indices],
    y=corr_indicators[sorted_indices],
    marker=dict(color='rgba(31, 119, 180, 0.8)', line=dict(color='DarkSlateGrey', width=1))
), row=2, col=1)

fig.update_xaxes(title_text='Country', showline=True, linewidth=2, linecolor='black', row=2, col=1,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))
fig.update_yaxes(title_text='Poverty centrality correlation in initial and final years of survey', showline=True, linewidth=2, linecolor='black', row=2, col=1,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))

fig.update_layout(height=800)

fig.show()

# Export the figure as a high-quality PDF
pio.write_image(fig, 'output/figure2.pdf', format='pdf', width=1200, height=800, scale=2)

## TABLE 1

We explore the empirical association between the Poverty Centrality and the long run changes in the censored headcount ratio of a poverty indicator by pooling data from all participating countries and constructing regression models in which the dependent variable is the change in the censored headcount ratio from the first to the last year of surveys for each poverty indicator and country. In these regressions our Poverty Centrality is used as an explanatory variable.


In column (1) of Table 1, we present a baseline regression model that includes only the dummy variables, which accounts for approximately 29% of the observed changes in the censored headcount ratio (Adjusted R^2=0.29). When we our poverty centrality index (column (2)), we get a significant improvement in the model's explanatory power (Adjusted R^2=0.37). More importantly, we find that more central indicators usually have larger decreases in the headcount ratio, as suggested by our analysis in the previous section.

In column (3), we include the initial censored headcount ratio of the indicator H_ci (t) as an additional variable, while column (4) adds the sum of the headcount ratios of all other indicators, excluding the one under consideration. Lastly, in column (5), we include both control variables in our regression model. Our results demonstrate that the negative and significant relationship between poverty centrality and long-run changes in headcount ratios persists even after adjusting for these controls. Additionally, we find that indicators with higher initial censored headcount ratios display larger decreases over time. The sum of the headcount ratios of also has a negative coefficient in the final model, though it is not statistically significant.



In [ ]:
## TABLE 1

# OLS regression examples
reg_baseline = smf.ols(formula='change_A ~ var_sample + country', data=TT_regression).fit(cov_type='HC1')
reg_centrality = smf.ols(formula='change_A ~ centrality_w_norm + var_sample + country', data=TT_regression).fit(cov_type='HC1')
reg_addiitonal_exp_A_int = smf.ols(formula='change_A ~ A_int + centrality_w_norm + var_sample + country', data=TT_regression).fit(cov_type='HC1')
reg_addiitonal_exp_A_avg = smf.ols(formula='change_A ~ A_avg_raw + centrality_w_norm + var_sample + country', data=TT_regression).fit(cov_type='HC1')
reg_addiitonal_exp = smf.ols(formula='change_A ~ A_int + A_avg_raw + centrality_w_norm + var_sample + country', data=TT_regression).fit(cov_type='HC1')


# Display regression results using stargazer
stargazer = Stargazer([reg_baseline, reg_centrality, reg_addiitonal_exp_A_int, reg_addiitonal_exp_A_avg, reg_addiitonal_exp])

# Customize stargazer output
stargazer.title('Change in headcount ratio models results')
stargazer.custom_columns(['(1)', '(2)', '(3)', '(4)', '(5)'], [1, 1, 1, 1, 1])
stargazer.significant_digits(3)
stargazer.show_model_numbers(False)
stargazer.dependent_variable_name('Change in headcount ratio per year')

# Customize the order of the variables
stargazer.covariate_order([
    'centrality_w_norm', 
    'A_int', 
    'A_avg_raw'
])

stargazer.rename_covariates({
    'A_int': 'Initial censored headcount ratio',
    'centrality_w_norm': 'Initial poverty centrality',
    'A_avg_raw': 'Initial headcount ratios of all other indicators',
})

# Render the HTML and save it to a file
html_output = stargazer.render_html()

# Save the HTML output to a file
with open('output/table1.html', 'w') as file:
    file.write(html_output)

# Display the HTML in Jupyter Notebook
display(HTML(html_output))


## TABLE 2

In Table 2, we utilize data for 15 countries that have an additional survey conducted between the initial and final years of the survey period. For these nations, we estimate the Poverty Space (and the corresponding Poverty Centrality) for all three survey years and compute the change in headcount ratio between the first and second, as well as the second and third survey years. This enables us to construct a new unbalanced sample which is then used to re-estimate our model. To maintain the restrictiveness of the model we also include year dummies in this analysis. The results, affirm that Poverty Centrality remains a significant and negative predictor of long-run changes in headcount ratios even within this more homogeneous sample.


In [ ]:
## TABLE 2

# OLS regression examples
reg_baseline = smf.ols(formula='change_A ~ var_sample + country + C(year)', data=TT_regression_3_period).fit(cov_type='HC1')
reg_centrality = smf.ols(formula='change_A ~ centrality_w_norm + var_sample + country + C(year)', data=TT_regression_3_period).fit(cov_type='HC1')
reg_addiitonal_exp_A_int = smf.ols(formula='change_A ~ A_int + centrality_w_norm + var_sample + country + C(year)', data=TT_regression_3_period).fit(cov_type='HC1')
reg_addiitonal_exp_A_avg = smf.ols(formula='change_A ~ A_avg_raw + centrality_w_norm + var_sample + country + C(year)', data=TT_regression_3_period).fit(cov_type='HC1')
reg_addiitonal_exp = smf.ols(formula='change_A ~ A_int + A_avg_raw + centrality_w_norm + var_sample + country + C(year)', data=TT_regression_3_period).fit(cov_type='HC1')


# Display regression results using stargazer
stargazer = Stargazer([reg_baseline, reg_centrality, reg_addiitonal_exp_A_int, reg_addiitonal_exp_A_avg, reg_addiitonal_exp])

# Customize stargazer output
stargazer.title('Change in headcount ratio models results')
stargazer.custom_columns(['Model 1', 'Model 2', 'Model 3', 'Model 4', 'Model 5'], [1, 1, 1, 1, 1])
stargazer.significant_digits(3)
stargazer.show_model_numbers(False)
stargazer.dependent_variable_name('Change in headcount ratio per year')

# Customize the order of the variables
stargazer.covariate_order([
    'centrality_w_norm', 
    'A_int', 
    'A_avg_raw'
])

stargazer.rename_covariates({
    'A_int': 'Initial censored headcount ratio',
    'centrality_w_norm': 'Initial poverty centrality',
    'A_avg_raw': 'Initial headcount ratios of all other indicators',
})

# Render the HTML and save it to a file
html_output = stargazer.render_html()

# Save the HTML output to a file
with open('output/table2.html', 'w') as file:
    file.write(html_output)

# Display the HTML in Jupyter Notebook
display(HTML(html_output))


## TABLE 3

Table 3, presents additional robustness results where we use an IV approach to estimate the relationship between poverty centrality and future changes in the censored headcount ratio. Endogeneity might because of unobserved factors influencing both Poverty Centrality and the changes in the censored headcount ratio. This scenario could lead to biased coefficient estimates.

The IV approach helps us tackle this issue by using an additional variable as an instrument. This variable should be related with the Poverty centrality, but uncorrelated with the error term of our model. Here, for each indicator in a country we define the instrumental variable as the respective poverty centrality of the same indicator in the country with the most similar poverty space in the initial time period (estimate through the coefficient of correlation between the edges of the Poverty Space). The logic behind this instrument is that countries with similar Poverty Spaces should also exhibit similar Poverty Centrality, but the exact circumstances (such as policy environment, demographic dynamics, etc.) leading to changes in the censored headcount ratio will not necessarily be the same, providing us with a source of exogenous variation.


The results consistently indicate that Poverty Centrality is a significant and negative predictor of the long-run changes in the headcount ratio. 


In [ ]:
# Cleaning data
TT_regression_cleaned = TT_regression.dropna(subset=['centrality_w_norm_inst'])

# OLS regression examples
reg_baseline = smf.ols(formula='change_A ~ var_sample + country', data=TT_regression_cleaned).fit(cov_type='HC1')

reg_1_stage_centrality = smf.ols(formula='centrality_w_norm ~ centrality_w_norm_inst + var_sample + country', data=TT_regression_cleaned).fit(cov_type='HC1')
TT_regression_cleaned['centrality_w_norm_predicted'] = reg_1_stage_centrality.predict(TT_regression_cleaned)
reg_centrality = smf.ols(formula='change_A ~ centrality_w_norm_predicted + var_sample + country', data=TT_regression_cleaned).fit(cov_type='HC1')

reg_1_stage_addiitonal_exp_A_int = smf.ols(formula='centrality_w_norm ~ A_int + centrality_w_norm_inst + var_sample + country', data=TT_regression_cleaned).fit(cov_type='HC1')
TT_regression_cleaned['centrality_w_norm_predicted'] = reg_1_stage_addiitonal_exp_A_int.predict(TT_regression_cleaned)
reg_addiitonal_exp_A_int = smf.ols(formula='change_A ~ A_int + centrality_w_norm_predicted + var_sample + country', data=TT_regression_cleaned).fit(cov_type='HC1')

reg_1_stage_addiitonal_exp_A_avg = smf.ols(formula='centrality_w_norm ~ A_avg_raw + centrality_w_norm_inst + var_sample + country', data=TT_regression_cleaned).fit(cov_type='HC1')
TT_regression_cleaned['centrality_w_norm_predicted'] = reg_1_stage_addiitonal_exp_A_avg.predict(TT_regression_cleaned)
reg_addiitonal_exp_A_avg = smf.ols(formula='change_A ~ A_avg_raw + centrality_w_norm_predicted + var_sample + country', data=TT_regression_cleaned).fit(cov_type='HC1')

reg_1_stage_addiitonal_exp = smf.ols(formula='centrality_w_norm ~ A_int + A_avg_raw + centrality_w_norm_inst + var_sample + country', data=TT_regression_cleaned).fit(cov_type='HC1')
TT_regression_cleaned['centrality_w_norm_predicted'] = reg_1_stage_addiitonal_exp.predict(TT_regression_cleaned)
reg_addiitonal_exp = smf.ols(formula='change_A ~ A_int + A_avg_raw + centrality_w_norm_predicted + var_sample + country', data=TT_regression_cleaned).fit(cov_type='HC1')

# Display regression results using stargazer
stargazer = Stargazer([reg_baseline, reg_centrality, reg_addiitonal_exp_A_int, reg_addiitonal_exp_A_avg, reg_addiitonal_exp])

# Customize stargazer output
stargazer.title('Change in headcount ratio models results')
stargazer.custom_columns(['Model 1', 'Model 2', 'Model 3', 'Model 4', 'Model 5'], [1, 1, 1, 1, 1])
stargazer.significant_digits(3)
stargazer.show_model_numbers(False)
stargazer.dependent_variable_name('Change in headcount ratio per year')

# Customize the order of the variables
stargazer.covariate_order([
    'centrality_w_norm_predicted', 
    'A_int', 
    'A_avg_raw'
])

stargazer.rename_covariates({
    'A_int': 'Initial censored headcount ratio',
    'centrality_w_norm_predicted': 'Initial poverty centrality',
    'A_avg_raw': 'Initial headcount ratios of all other indicators',
})

# Calculate first-stage F-statistics
first_stage_models = [reg_1_stage_centrality, reg_1_stage_addiitonal_exp_A_int, reg_1_stage_addiitonal_exp_A_avg, reg_1_stage_addiitonal_exp]
f_statistics = []
for model in first_stage_models:
    f_test = model.f_test("centrality_w_norm_inst = 0")
    f_statistics.append(f_test.fvalue)  # Handle as float directly

# Render the HTML
html_output = stargazer.render_html()

# Add F-statistics row to the HTML
html_output = html_output.replace(
    '</table>',
    '<tr><td>First-stage F-statistic</td><td></td>' +
    ''.join(f'<td>{f_stat:.3f}</td>' for f_stat in f_statistics) +
    '</tr></table>'
)

# Save the HTML output to a file
with open('output/table3.html', 'w') as file:
    file.write(html_output)

# Display the HTML in Jupyter Notebook
display(HTML(html_output))


#  PPI ANALYSIS
# (Figure 3)

In the empirical analysis we found that the Poverty Space is predictive for the future changes in the censored headcount ratio of a poverty indicator.

Motivated by this, here we use the Poverty Space in the Policy Priority Inference (PPI) tool. The
PPI is a computational framework created to understand the complexity of policy prioritization and to support governments who wish to distribute transformative resources across numerous policy issues with the aim of reaching specific development goals.

As an example, we use Ethiopia.

### Model initialization

PPI offers two distinct methods for analyzing multidimensional poverty. First, the “retrospective” analysis seeks to understand past dynamics by leveraging historical data from an initial survey year, revealing how poverty indicators and policy priorities (budget allocations for an indicator) have evolved over a certain observation period to reach observed values in a final year. This analysis can infer an optimal spillover rate (the spillover rate that best fits the data) and other model parameters (e.g., α_ci and α_ci^'). Understanding these parameters is essential as spillover rates can vary widely or modestly between dimensions and countries, influencing policy intervention outcomes. Second, the “prospective” analysis uses historical insights to understand how changes in the spillover rate or the Poverty Space could affect multidimensional poverty dynamics over a future period. This approach guides policymakers on which poverty indicators to prioritize and when, helping to inform effective intervention strategies for future improvements.

In the retrospective analysis, we use national data on poverty indicators from the first survey year (2011) and the last survey year (2019), government expenditure data (per capita in constant USD) from 2011 to 2019, data on the quality of law and monitoring from the World Bank’s World Governance Indicators for 2011 (the quality of law is approximated with the rule of law variable, whereas the quality of monitoring with the control of corruption), and the estimated Poverty Space for 2011. We estimate the model parameters and find the optimal spillover rate by calibrating the model with spillover rates ranging from 0 to 1 (in increments of 0.05) and calculating the goodness of fit. The rate providing the best fit on average is our optimal spillover rate. See Appendix 4 for more details on the data cleaning and calibration procedures for our PPI analysis.

We then conduct a prospective analysis until 2030 using data from the final survey year (2019) and Poverty Space data for the same year. We assume that the rule of law and quality of monitoring remain at the 2019 level and that government expenditure remains at its average value from 2011 to 2019. In this analysis, we again vary the spillover rate from 0 to 1 to perturb the Poverty Space structure and assess how poverty interlinkages could impact multidimensional poverty dynamics.

### Findings: 

Figure 3 details the impact of the Poverty Space on the inferred PPI simulations.

First, in Figure 3a we visualize how varying the spillover rate impacts PPI’s estimated MPI for Ethiopia in 2030 (black line). The blue horizontal dashed line represents Ethiopia’s MPI in 2019 (0.567, the beginning of the prospective simulation), while the red vertical line marks the optimal spillover rate identified in the retrospective analysis. Our analysis finds this value to be 0.7, indicating that spillovers could heavily influence Ethiopia’s multidimensional poverty dynamics. If the spillover rate remains at this level, we project Ethiopia’s MPI to decrease by 0.103 units by 2030 (to 0.464). Conversely, if the spillover rate drops to 0, the MPI would fall by only 0.036 units (to 0.531). An increase in the spillover rate to 1 (implying that the spillover network matches the Poverty Space) would result in a 0.109 unit decline in MPI (to 0.458). Interestingly, our results suggest that Ethiopia’s multidimensional poverty dynamics are more responsive to a reduction in the poverty rates when the MPI increases compared to an increase in the spillover rate above the inferred optimal value. These findings suggest that bottlenecks in spillovers between indicators could substantially affect poverty reduction in Ethiopia.

Next, in Figure 3 b, we show the average spillovers (between 2021 and 2030) received by each poverty indicator during the prospective analysis for various values of  r^c as a function of Poverty Centrality. We observe a nearly perfect correlation between these two variables for each choice of  r^c. This indicates that Poverty Centrality effectively reflects the spillovers in PPI simulations. Importantly, it suggests that while Poverty Centrality can identify which indicators are most likely to receive spillovers, the actual volume of spillovers is dependent on the rate   r^c . Thus, Poverty Centrality serves as a relative metric, providing insights into the potential distribution of spillovers among indicators, but the magnitude of these spillovers varies with the spillover rate.

Moving on to Figure 3 c, we illustrate the censored headcount ratio of each indicator as a function of the spillover rate. This analysis allows us to see how each poverty indicator’s censored headcount ratio changes as the spillover rate varies. Indicators with the lowest and highest values for the censored headcount ratio show the least susceptibility to changes in the spillover rate, indicating that their the censored headcount ratio remains relatively stable even as  r^c varies. In contrast, indicators with intermediate levels of the censored headcount ratio appear the most susceptible, showing greater fluctuations with changes in  r^c.

Figure 3 d provides a more detailed view by plotting the changes in the censored headcount ratio for each indicator as a function of their 2019 levels under different spillover rates. Indeed, two indicators with intermediate levels of CHR in 2019 (Schooling and Drinking Water) display the largest change due to changes in the spillover rate. This result suggests that the responsiveness of indicators with intermediate censored headcount ratio levels to changes in spillover rates might be due to shifts in policy priority driven by spillover effects, at least in Ethiopia. When spillovers are present, resources and attention may be redistributed towards these intermediate indicators, thereby amplifying their susceptibility to changes in the spillover rate. This underscores the importance of considering how policy priorities and resource allocations may shift due to spillover effects. By identifying which indicators are most responsive to these shifts, policymakers can better target their efforts to maximize the impact of resource allocation and intervention strategies, ensuring that critical areas receive the necessary support to reduce poverty effectively.

The results are averaged across 100 PPI simulations.

In [ ]:

# Initialize variables
r_values = np.arange(0, 1.05, 0.05)
gof = []
# Load and filter data
country = 'eth'
filtered_df = TT_FINAL[TT_FINAL['country'] == country].copy()

# Create initial data frame
data = pd.DataFrame()
data['seriesCode'] = pd.Series(filtered_df['var_sample'].unique())
data['goal'] = 1
data['target'] = 1
data['type'] = 0
data['countryCode'] = country
mapping_dict = filtered_df.set_index('var_sample')['full_names'].to_dict()
data['seriesName'] = data['seriesCode'].map(mapping_dict)

years = pd.to_numeric(filtered_df['year'], errors='coerce')
unique_years = np.sort(years.unique())

filtered_df_1 = filtered_df[years == unique_years[0]]
mapping_dict = filtered_df_1.set_index('var_sample')['A_int'].to_dict()
data['1'] = 1 - data['seriesCode'].map(mapping_dict)
mapping_dict = filtered_df_1.set_index('var_sample')['centrality_w_norm'].to_dict()
data['PC_1'] = data['seriesCode'].map(mapping_dict)

filtered_df_2 = filtered_df[years == unique_years[1]]
mapping_dict = filtered_df_2.set_index('var_sample')['A_int'].to_dict()
data['2'] = 1 - data['seriesCode'].map(mapping_dict)
mapping_dict = filtered_df_2.set_index('var_sample')['centrality_w_norm'].to_dict()
data['PC_2'] = data['seriesCode'].map(mapping_dict)

filtered_df_3 = filtered_df[years == unique_years[2]]
mapping_dict = filtered_df_3.set_index('var_sample')['A_int'].to_dict()
data['3'] = 1 - data['seriesCode'].map(mapping_dict)
mapping_dict = filtered_df_3.set_index('var_sample')['centrality_w_norm'].to_dict()
data['PC_3'] = data['seriesCode'].map(mapping_dict)

data['instrumental'] = [0, 1, 0, 0, 1, 1, 1, 1, 1, 1]
mask = np.array(data['instrumental']) == 1

data['colors'] = ['#E5243B', '#DDA63A', '#4C9F38', '#C5192D', '#FF3A21', '#26BDE2', '#FCC30B', '#A21942', '#FD6925', '#DD1367']
indis_index = dict([(code, i) for i, code in enumerate(data.seriesCode)]) # used to build the network matrix

# Load government expenditure data
data_exp = pd.read_excel('data/general-government-data.xlsx', sheet_name='cleaned_data')
data_wgi = pd.read_excel('data/wgi-data.xlsx', sheet_name='cleaned_data')

# Inference parameters
qm = float(data_wgi.loc[data_wgi['year'] == 2011, 'qm'].values[0]) # Governance param 1
rl = float(data_wgi.loc[data_wgi['year'] == 2011, 'rl'].values[0]) # Governance param 2
T = 54



years_exp = [column_name for column_name in data_exp.columns if str(column_name).isnumeric()]
periods = len(years_exp)
t = int(T / periods)
div = T / len(years_exp)
new_rows = []
for index, row in data_exp.iterrows():
    new_row = [row.sdg]
    for year in years_exp:
        new_row += [int(row[year]) / div for i in range(t)]
    new_rows.append(new_row)

df_exp = pd.DataFrame(new_rows, columns=['sdg'] + list(range(T)))
Bs = df_exp.values[:, 1:] # disbursement schedule (assumes that the expenditure programmes are properly sorted)

is_instrumental = dict(zip(data.seriesCode, data.instrumental == 1))
rel_dict = dict([(code, []) for code in data.seriesCode if is_instrumental[code]])
for index, row in data.iterrows():
    if row.seriesCode in rel_dict:
        rel_dict[row.seriesCode].append(row.type)

n_cols = max([len(value) for value in rel_dict.values()])

M = [['' for i in range(n_cols + 1)] for code in rel_dict.values()]
for i, items in enumerate(rel_dict.items()):
    sdg, indis = items
    M[i][0] = sdg
    for j, indi in enumerate(indis):
        M[i][j + 1] = indi

df_rel = pd.DataFrame(M, columns=['seriesCode'] + list(range(n_cols)))
B_dict = {} # PPI needs the relational table in the form of a Python dictionary
for index, row in df_rel.iterrows():
    B_dict[indis_index[row.seriesCode]] = [programme for programme in row.values[1:] if str(programme) != 'nan']

R = data.instrumental # instrumental indicators

# Calibration
parallel_processes = 6 # number of cores to use
threshold = 0.8 # the quality of the calibration (I choose a medium quality for illustration purposes)
low_precision_counts = 100 # number of low-quality iterations to accelerate the calibration

# Data for inference
I0 = data['1']
IF = data['3']
years_cols = [col for col in data.columns if col.isnumeric()]
successRates = np.sum(data[years_cols].values[:, 1:] > data[years_cols].values[:, :-1], axis=1) / (len(years_cols) - 1)
successRates[successRates == 0] = .1
successRates[successRates == 1] = .9

# Loop over r values and calculate x
for r in r_values:
    # Update the spillover network A with the current r value
    A = PHI_NORMALIZED[f'phi_{country}_{unique_years[0]}'].copy()
    A = r * A.T

    parameters = ppi.calibrate(I0, IF, successRates, A=A, R=R, qm=qm, rl=rl, Bs=Bs, B_dict=B_dict,
                               T=T, threshold=threshold, parallel_processes=parallel_processes, verbose=False,
                               low_precision_counts=low_precision_counts)
    df_params = pd.DataFrame(parameters[1:], columns=parameters[0])
    
    error = np.mean(df_params['GoF_alpha'].values.astype(float))
    gof.append(error)
    
    print(f"r value = {r} and GoF = {error}")
    
# Find the r value that corresponds to the maximum GoF
max_gof_index = np.argmax(gof)
r_opt = r_values[max_gof_index]    
    
## SIMULATE FIGURE 3 a and b
# retrospective
A = PHI_NORMALIZED[f'phi_{country}_{unique_years[0]}'].copy()
A = r_opt * A.T
parameters = ppi.calibrate(I0, IF, successRates, A=A, R=R, qm=qm, rl=rl, Bs=Bs, B_dict=B_dict,
                               T=T, threshold=threshold, parallel_processes=parallel_processes, verbose=False,
                               low_precision_counts=low_precision_counts)
df_params = pd.DataFrame(parameters[1:], columns=parameters[0])

# prospective parameters
qm = float(data_wgi.loc[data_wgi['year'] == 2019, 'qm'].values[0]) # Governance param 1
rl = float(data_wgi.loc[data_wgi['year'] == 2019, 'rl'].values[0]) # Governance param 2
sample_size = 100 # number of Monte Carlo simulations
Imax = np.ones(len(IF))
Imin = np.zeros(len(IF))
Tx = 66
Bs_x = np.zeros(Tx) + np.mean(Bs)

goals = data['goal'].values.astype(float)
alphas = df_params.alpha.values.astype(float)
alphas_prime = df_params.alpha_prime.values.astype(float)
betas = df_params.beta.values.astype(float)

CensoredHeadcountRatio = np.full((len(goals), len(r_values)), np.nan)
EstimatedSpillovers = np.full((len(goals), len(r_values)), np.nan)
EstimatedContributions = np.full((len(goals), len(r_values)), np.nan)
EstimatedBenefits = np.full((len(goals), len(r_values)), np.nan)
EstimatedAllocations = np.full((len(goals), len(r_values)), np.nan)
EstimatedGrowth = np.full((len(goals), len(r_values)), np.nan)

for r_index, r in enumerate(r_values):
    # Update the spillover network A with the current r value
    A = PHI_NORMALIZED[f'phi_{country}_{unique_years[2]}'].copy()
    A = r * A.T

    outputs = []
    for sample in range(sample_size):
        output = ppi.run_ppi(IF, alphas, alphas_prime, betas, A=A, R=R, qm=qm, rl=rl,Imax=Imax, Imin=Imin, Bs=Bs_x, B_dict=B_dict, T=Tx, G=goals, seed=sample)
        outputs.append(output)

    tsI, tsC, tsF, tsP, tsS, tsG = zip(*outputs)
    tsI_hat = np.mean(tsI, axis=0)
    tsC_hat = np.mean(tsC, axis=0)
    tsF_hat = np.mean(tsF, axis=0)
    tsP_hat = np.mean(tsP, axis=0)        
    tsS_hat = np.mean(tsS, axis=0)
    tsG_hat = np.mean(tsG, axis=0) 

    # Set CensoredHeadcountRatio for each r (column) as the last column of tsI_hat
    CensoredHeadcountRatio[:, r_index] = 1-tsI_hat[:, -1]
    
    # Set the value of EstimatedSpillovers for each r as the sum of each row of tsS_hat
    EstimatedSpillovers[:, r_index] = np.mean(tsS_hat, axis=1)
    EstimatedContributions[mask, r_index] = np.mean(tsC_hat, axis=1)
    EstimatedBenefits[mask, r_index] = np.mean(tsF_hat, axis=1)
    EstimatedAllocations[mask, r_index] = np.mean(tsP_hat, axis=1)
    EstimatedGrowth[:, r_index] = np.mean(tsG_hat, axis=1)
    print(r)
    

# Calculate the sum of CensoredHeadcountRatio across all goals
sum_CensoredHeadcountRatio = np.sum(CensoredHeadcountRatio, axis=0)



# Assuming you have the `1-IF` values and the `CensoredHeadcountRatio` for each indicator
one_minus_IF = 1 - IF  # Calculate 1-IF if it's not already provided

## FIGURE

# Create a 2x2 subplot figure
fig = make_subplots(rows=2, cols=2, subplot_titles=('a', 'b', 'c', 'd'))

# Define a list of colors, line styles, and markers
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
line_styles = ['solid', 'dash', 'dot', 'dashdot']
markers = ['circle', 'diamond', 'triangle-up', 'square']

# Add sum of CensoredHeadcountRatio to first subplot
fig.add_trace(go.Scatter(
    x=r_values,
    y=sum_CensoredHeadcountRatio,
    mode='lines',
    name='MPI',
    line=dict(color='black', width=9),
    showlegend=False
), row=1, col=1)

# Add vertical and horizontal lines to the first subplot
fig.add_shape(type="line", x0=r_opt, y0=0, x1=r_opt, y1=np.sum(1-IF), xref='x', yref='y', line=dict(color="maroon", width=3), row=1, col=1)
fig.add_shape(type="line", x0=r_values[0], y0=np.sum(1-IF), x1=r_values[-1], y1=np.sum(1-IF), xref='x', yref='y', line=dict(color="blue", width=3, dash="dash"), row=1, col=1)

# Add each goal's CensoredHeadcountRatio to the second subplot
for i in range(len(goals)):
    fig.add_trace(go.Scatter(
        x=r_values,
        y=CensoredHeadcountRatio[i, :],
        mode='lines+markers',
        name=data['seriesName'][i],
        line=dict(color=colors[i % len(colors)], width=1.5, dash=line_styles[i % len(line_styles)]),
        marker=dict(symbol=markers[i % len(markers)], size=6),
        showlegend=True
    ), row=2, col=1)

# Add vertical line to the second subplot
fig.add_shape(type="line", x0=r_opt, y0=-0.01, x1=r_opt, y1=np.max(CensoredHeadcountRatio)*1.1, xref='x', yref='y', line=dict(color="maroon", width=3), row=2, col=1)

# Sort indicators by 1-IF values
sorted_indices = np.argsort(one_minus_IF)
sorted_indicators = np.array(data['seriesName'])[sorted_indices]
sorted_one_minus_IF = one_minus_IF[sorted_indices]
sorted_CensoredHeadcountRatio = CensoredHeadcountRatio[sorted_indices, :]

# Bar charts in the fourth subplot
r_values_to_plot = [0, 0.3, r_opt, 1]
for r_index, r in enumerate(r_values_to_plot):
    r_value_index = np.where(np.isclose(r_values, r))[0][0]
    y_values = sorted_CensoredHeadcountRatio[:, r_value_index] - sorted_one_minus_IF
    fig.add_trace(go.Bar(
        x=sorted_indicators,
        y=y_values,
        name=f'r = {r}',
        marker=dict(color=colors[r_index % len(colors)]),
        showlegend=True
    ), row=2, col=2)

# Scatter plots in the third subplot
for r_index, r in enumerate(r_values_to_plot):
    r_value_index = np.where(np.isclose(r_values, r))[0][0]
    correlation = np.corrcoef(data['PC_3'], EstimatedSpillovers[:, r_value_index])[0, 1]
    fig.add_trace(go.Scatter(
        x=data['PC_3'],
        y=EstimatedSpillovers[:, r_value_index],
        mode='markers',
        name=f'r = {r}, Corr = {correlation:.2f}',
        marker=dict(size=8, symbol=markers[r_index % len(markers)], line=dict(width=2), color=colors[r_index % len(colors)]),
        showlegend=True
    ), row=1, col=2)

# Update layout
fig.update_layout(
    plot_bgcolor='white',
    showlegend=True,
    legend=dict(orientation='h', yanchor='bottom', y=1.1, xanchor='center', x=0.5),
    margin=dict(l=40, r=40, t=40, b=40)
)

# Update axes for each subplot
axis_params = dict(showline=True, showgrid=True, showticklabels=True, linecolor='black', linewidth=2, ticks='outside', tickfont=dict(family='Arial', size=12, color='black'), gridcolor='lightgrey')
fig.update_xaxes(**axis_params, title_text='Spillover rate', tickvals=[0, 0.2, 0.4, 0.6, 0.8, 1], row=1, col=1)
fig.update_yaxes(**axis_params, title_text='MPI', range=[0.4, np.max(sum_CensoredHeadcountRatio) * 1.1], row=1, col=1)
fig.update_xaxes(**axis_params, title_text='Spillover rate', tickvals=[0, 0.2, 0.4, 0.6, 0.8, 1], row=2, col=1)
fig.update_yaxes(**axis_params, title_text='CHR', row=2, col=1)
fig.update_xaxes(**axis_params, title_text='Indicators', tickvals=list(range(len(sorted_indicators))), row=2, col=2)
fig.update_yaxes(**axis_params, title_text='Change in CHR', row=2, col=2)
fig.update_xaxes(**axis_params, title_text='Poverty Centrality', row=1, col=2)
fig.update_yaxes(**axis_params, title_text='Estimated Spillovers', row=1, col=2)

# Export the figure as a high-quality PDF
width_cm = 20  # desired width in cm
height_cm = 21  # desired height in cm
width_px = width_cm * 37.7953
height_px = height_cm * 37.7953
fig.write_image("output/figure3.pdf", format='pdf', width=width_px, height=height_px, scale=3)

fig.show()



# APPENDIX 3 FIGURE A3.1

This figure shows the median poverty centrality by income group. We find that the patterns by income group are similar to the overall patterns discovered in Figure 2b.

In [ ]:
# PLOT MEDIAN POVERTY CENTRALITY BY INCOME GROUP

file_path = 'data/OGHIST.xlsx'
sheet_name = 'Country Analytical History'

df_region = pd.read_excel(file_path, sheet_name=sheet_name)
df_region['Code'] = df_region['Code'].str.lower()

TT_regression_plot = TT_regression.copy()
TT_regression_plot['year'] = 2000 + TT_regression_plot['year'].astype(int)
TT_regression_plot = TT_regression_plot.merge(df_region[['Code','Year', 'Group']], left_on=['country', 'year'], right_on=['Code','Year'], how='left')

# Get unique regions
unique_regions = TT_regression_plot['Group'].unique()
unique_regions = unique_regions[unique_regions != 'H'] # Drop high income

# Mapping unique region codes to their full names
region_titles = {
    'UM': 'Upper-middle income',
    'LM': 'Lower-middle income',
    'L': 'Low income'
}

# Create a subplot with a column for each unique region
fig = make_subplots(rows=1, cols=len(unique_regions), shared_yaxes=True, subplot_titles=[region_titles[region] for region in unique_regions])

# Loop through each region and create the bar chart
for i, region in enumerate(unique_regions):
    # Filter the data for the current region
    region_data = TT_regression_plot[TT_regression_plot['Group'] == region]
    median_indicator = region_data.groupby('full_names')['centrality_w_norm_final'].median().reset_index()
    
    # Merge with median_indicator_sorted_general to get the correct order
    median_indicator_sorted = median_indicator.merge(median_indicator_sorted_general[['full_names']], on='full_names', how='right')
    
    # Add the bar chart to the subplot
    fig.add_trace(go.Bar(
        x=median_indicator_sorted['centrality_w_norm_final'],
        y=median_indicator_sorted['full_names'],
        orientation='h',
        marker=dict(color='rgba(31, 119, 180, 0.8)', line=dict(color='DarkSlateGrey', width=1)),
        showlegend=False
    ), row=1, col=i+1)

    # Update x and y axes for the subplot
    fig.update_xaxes(title_text='Median Poverty Centrality', showline=True, linewidth=2, linecolor='black',
                     titlefont=dict(color='black'), tickfont=dict(color='black'), row=1, col=i+1)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True,
                     titlefont=dict(color='black'), tickfont=dict(color='black'), row=1, col=i+1)

# Update layout to set all text to black
fig.update_layout(
    height=600,
    width=1500,
    title_text="Median Poverty Centrality by Income Group",
    title_font=dict(color='black'),
    font=dict(color='black'),
    paper_bgcolor='white',  # Set background color to white
    plot_bgcolor='white',  # Set plot area background color to white
)



# Show the plot
fig.show()

# Export the figure as a high-quality PDF
pio.write_image(fig, 'output/figureA31.pdf', format='pdf', width=1200, height=500, scale=2)

# APPENDIX 3 FIGURE A3.2

This figure shows the median poverty centrality by region. We find that the patterns by region are different to the overall patterns discovered in Figure 2b.

In [ ]:
# Load the data
file_path = 'data/OGHIST.xlsx'
sheet_name = 'region'
df_region = pd.read_excel(file_path, sheet_name=sheet_name)
df_region['Code'] = df_region['Code'].str.lower()

# Merge with TT_regression to add the Region information
TT_regression_plot = TT_regression.copy()
TT_regression_plot = TT_regression_plot.merge(df_region[['Code', 'Region']], left_on='country', right_on='Code', how='left')

# Get unique regions
unique_regions = TT_regression_plot['Region'].unique()

# Create a subplot with a column for each unique region
fig = make_subplots(rows=1, cols=len(unique_regions), shared_yaxes=True, subplot_titles=unique_regions)

# Loop through each region and create the bar chart
for i, region in enumerate(unique_regions):
    # Filter the data for the current region
    region_data = TT_regression_plot[TT_regression_plot['Region'] == region]
    median_indicator = region_data.groupby('full_names')['centrality_w_norm_final'].median().reset_index()
    
    # Merge with median_indicator_sorted_general to get the correct order
    median_indicator_sorted = median_indicator.merge(median_indicator_sorted_general[['full_names']], on='full_names', how='right')
    
    # Add the bar chart to the subplot
    fig.add_trace(go.Bar(
        x=median_indicator_sorted['centrality_w_norm_final'],
        y=median_indicator_sorted['full_names'],
        orientation='h',
        marker=dict(color='rgba(31, 119, 180, 0.8)', line=dict(color='DarkSlateGrey', width=1)),
        showlegend=False
    ), row=1, col=i+1)

    # Update x and y axes for the subplot
    fig.update_xaxes(title_text='Median Poverty Centrality', showline=True, linewidth=2, linecolor='black',
                     titlefont=dict(color='black'), tickfont=dict(color='black'), row=1, col=i+1)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True,
                     titlefont=dict(color='black'), tickfont=dict(color='black'), row=1, col=i+1)

# Update layout
# Update layout to set all text to black
fig.update_layout(
    height=600,
    width=1500,
    title_text="Median Poverty Centrality by Region",
    title_font=dict(color='black'),
    font=dict(color='black'),
    paper_bgcolor='white',  # Set background color to white
    plot_bgcolor='white',  # Set plot area background color to white
)
# Show the plot
fig.show()

# Export the figure as a high-quality PDF
pio.write_image(fig, 'output/figureA32.pdf', format='pdf', width=1200, height=500, scale=2)


## APPENDIX 2 FIGURE A2.1.

Figure A2.1. Reproduces Figure 2 from the main manuscript, describing the general patterns of the Poverty Space among countries, just instead of using a threshold of being poor in at least one dimension for the definition of multidimensional poverty, we use a threshold of being poor in at least 3 dimensions.

In [ ]:
## CLEAR DATA: ESTIMATE THE POVERTY SPACE & POVERTY CENTRALITY


var_weights = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
variables = ['d_cm_01', 'd_nutr_01', 'd_satt_01', 'd_educ_01', 'd_elct_01',
             'd_wtr_01', 'd_sani_01', 'd_hsg_01', 'd_ckfl_01', 'd_asst_01']
dimensions = [1, 1, 2, 2, 3, 3, 3, 3, 3, 3]
full_variable_names = ['Child mortality', 'Nutrition', 'School attendance', 'Schooling',
                       'Electricity', 'Drinking water', 'Sanitation', 'Housing',
                       'Cooking fuel', 'Assets']

threshold = 0.3

# In Python, using glob, you can search for all CSV files in the current working directory

search_pattern = os.path.join(os.getcwd(), 'data', '*.csv')
csv_files = glob.glob(search_pattern)

PHI_NORMALIZED = {}
list_names = {}
list_short_names = {}

Correlation_matrix = np.nan * np.zeros((len(csv_files), len(variables)))

                                       
columns = ['country', 'year', 'MPI', 'var_sample', 'A_int', 'A_avg', 'A_avg_raw', 'A_int_rank', 'centrality_w_norm', 'centrality_w_norm_rank', 'full_names']
TT_FINAL = pd.DataFrame(columns=columns)


for i in range(len(csv_files)):

    # Get the file name from the path
    filename = os.path.basename(csv_files[i])

    # Extract country code and year from the filename
    ctry = filename[:3]
    yyr = filename[-6:-4]  # Adjust according to your file naming convention if needed

    # Read the data
    TT = pd.read_csv(csv_files[i])

    # Create the deprivation matrix
    XX = pd.DataFrame()
    jj = []

    for j, var in enumerate(variables):
        if var in TT.columns:
            X = TT[var]
            if X.isna().mean() < 1:  # Check if the column is not entirely NaN
                XX = pd.concat([XX, X], axis=1)
            else:
                jj.append(j)  # Append j+1 to maintain MATLAB's 1-based indexing

    # Assuming previous steps have been correctly executed

    # Convert XX back to DataFrame if not already, to simplify handling NaN values
    # This step is precautionary; XX should already be a DataFrame after previous operations
    XX = pd.DataFrame(XX)

    # Filter out rows with any NaN values across all columns in XX
    # Also filter corresponding rows in the weights array
    valid_rows = ~XX.isnull().any(axis=1)
    XX_filtered = XX[valid_rows]

    weights = TT['weight'].values  # Extracting weight values from the DataFrame
    weights_sample = np.array(var_weights)  # Ensure var_weights is a numpy array for easy manipulation

    # Adjust weights_sample based on jj, ensuring jj is zero-based
    weights_sample = np.delete(weights_sample, jj)

    weights_filtered = weights[valid_rows]

    var_sample = [var for i, var in enumerate(variables) if i not in jj]
    full_names = [name for i, name in enumerate(full_variable_names) if i not in jj]


    
    # Normalize the sample weights
    weights_sample_normalized = weights_sample / sum(weights_sample)

    # Calculate the weighted deprivation matrix G
    # Note: Element-wise multiplication of weights_sample_normalized with each column in XX_filtered
    G = XX_filtered.multiply(weights_sample_normalized, axis='columns')

    # Calculate Z based on the threshold condition
    Z = np.where(G.sum(axis=1) > threshold, 1, 0) if threshold == 0 else np.where(G.sum(axis=1) >= threshold, 1, 0)

    # Calculate the final index H
    H = np.sum(weights_filtered * Z) / np.sum(weights_filtered)    


    # Ensure G is a numpy array for these operations, if not already
    G = G.to_numpy() if hasattr(G, 'to_numpy') else G

    # Recreate G_k from G
    G_k = np.copy(G)

    # Initialize xx with zeros with the correct dimensions
    # Note: Adjusting approach to ensure dimensions match expectations for numpy operations
    xx = np.zeros((np.sum(1 - Z), G_k.shape[1]))

    # Update G_k directly with numpy where applicable
    # We need to ensure we're updating rows where Z is 0 with xx, but this approach requires boolean indexing or equivalent in numpy
    not_Z_rows = np.where(Z == 0)[0]  # Get indices of rows where Z is 0

    # Since direct assignment as attempted can mismatch dimensions, we ensure alignment before attempting to replace
    # However, given xx aims to replace all not_Z_rows with zeros, we can simplify by directly setting zeros without using xx
    G_k[not_Z_rows, :] = 0  # Directly set rows where Z is 0 to zeros, aligning with intention

    # Proceed with adjusted calculations
    # Note: The calculations for W, A, A_est, and MPI should then ensure they are using numpy operations as well

    W_filtered = weights_filtered.reshape(-1, 1)  # Ensure weights_filtered is column vector for matrix operations
    A = (W_filtered * G_k) / np.sum(weights_filtered[Z == 1])

    A_est = np.sum(A, axis=0)

    MPI = np.sum(A_est * H)

    A_est *= H  # Update A_est by multiplying it with H

    # Create weighted networks
    Z_indices = np.where(Z)[0]  # Find indices where Z is True
    G_z = XX_filtered.iloc[Z_indices, :].to_numpy()  # Filter G_z based on Z

    # Ensure 'weights' and 'Z' are numpy arrays for the operations
    weights_final = np.array(weights_filtered)
    Z = np.array(Z, dtype=bool)
    A = np.array(A)  # Assuming 'A' is already correctly shaped and matches 'Z'

    # Select rows from 'A' where 'Z' is True
    A_filtered = A[Z, :]

    # Calculate the dot product of 'weights' transposed and 'Z' (which should be a scalar if both are vectors)
    weights_Z_product = np.dot(weights_final.T, Z)

    # Scale 'A_filtered' by 'weights_Z_product'
    M_w = weights_Z_product * A_filtered ## NOT CALCULATED CORRECTLY!!!!

    # Remove columns where the sum across rows in G_z is 0
    xx = np.sum(G_z > 0, axis=0)
    jj = np.where(xx == 0)[0]

    # Update var_sample and full_names by removing indices jj
    var_sample = np.delete(np.array(var_sample), jj).tolist()
    full_names = np.delete(np.array(full_names), jj).tolist()

    # Update G_z, M_w, and M_u by removing columns jj
    G_z = np.delete(G_z, jj, axis=1)
    M_w = np.delete(M_w, jj, axis=1)

    # Create similarity networks
    PHI_w_int = np.dot(M_w.T, G_z)  # Calculate initial similarity matrix
    PHI_w_norm = np.full(PHI_w_int.shape, np.nan)  # Initialize PHI_w_norm with NaN

    # Normalize PHI_w_int to create PHI_w_norm
    for k in range(PHI_w_int.shape[0]):
        for j in range(PHI_w_int.shape[1]):
            PHI_w_norm[k, j] = PHI_w_int[k, j] / PHI_w_int[j, j]

    # Adjust PHI_w_norm by subtracting the identity matrix scaled by PHI_w_norm
    PHI_w_norm = PHI_w_norm - np.eye(PHI_w_norm.shape[0]) * PHI_w_norm

    D, V = np.linalg.eig(PHI_w_norm)

    # Find the index of the eigenvalue with the maximum magnitude
    z = np.argmax(D)

    # Extract the corresponding eigenvector and compute the centrality measure
    centrality_w_norm = V[:, z]

    # Normalize the centrality measure so that its sum equals 1
    centrality_w_norm /= np.sum(centrality_w_norm)
    
    for j in range(len(variables)):
        
        try:
            index = var_sample.index(variables[j])
        except ValueError:
            
            continue
        
        Phi_reduced = np.delete(PHI_w_norm, index, axis=0).copy()  # Remove row
        Phi_reduced = np.delete(Phi_reduced, index, axis=1)  # Remove column
        centrality_reduced = np.delete(centrality_w_norm, index).copy()
        
        D, V = np.linalg.eig(Phi_reduced)

        # Find the index of the eigenvalue with the maximum magnitude
        z = np.argmax(D)

        # Extract the corresponding eigenvector and compute the centrality measure
        centrality_w_norm_reduced = V[:, z]

        # Normalize the centrality measure so that its sum equals 1
        centrality_w_norm_reduced /= np.sum(centrality_w_norm_reduced)
        centrality_reduced /= np.sum(centrality_reduced)
        
        
        # Calculate the correlation and assign it to the Correlation_matrix
        corr_matrix = np.corrcoef(centrality_w_norm_reduced, centrality_reduced)
        # corr_matrix[0, 1] or corr_matrix[1, 0] contains the correlation coefficient
        Correlation_matrix[i, j] = corr_matrix[0, 1]

  
    

    # Assuming ctry, yyr, MPI, var_sample, centrality_w_norm, A_est, PHI_w_norm, and full_names are defined

    num_vars = len(var_sample)

    # 1. Replicating values
    country = np.repeat(ctry, num_vars)
    year = np.repeat(yyr, num_vars)
    MPI_repeated = np.repeat(MPI, num_vars)

    # 2. Sorting and ranking centrality_w_norm
    centrality_w_norm_rank = rankdata(-centrality_w_norm, method='ordinal')


    # Sorting A_int and ranking
    A_int = A_est.T
    A_int = np.delete(A_int, jj)  # Remove indices as specified by jj
    A_int_rank = rankdata(-A_int, method='ordinal')


    # 3. Calculating Averages
    A_avg = PHI_w_norm @ A_int
    PHI_ones = np.ones_like(PHI_w_norm)
    PHI_ones -= np.eye(len(A_avg))
    A_avg_raw = PHI_ones @ A_int

    # 4. Creating the results table
    # Note: var_sample, centrality_w_norm, and full_names should be lists or arrays of matching length
    TT_results = pd.DataFrame({
        'country': country,
        'year': year,
        'MPI': MPI_repeated,
        'var_sample': var_sample,
        'A_int': A_int,
        'A_avg': A_avg,
        'A_avg_raw': A_avg_raw,
        'A_int_rank': A_int_rank,  # Adjust for zero-based indexing in Python
        'centrality_w_norm': centrality_w_norm,
        'centrality_w_norm_rank': centrality_w_norm_rank,
        'full_names': full_names
    })

    # Dynamically setting values in dictionaries using the current value of i
    PHI_NORMALIZED[f'phi_{ctry}_{yyr}'] = PHI_w_norm
    list_names[f'variable_names_{ctry}_{yyr}'] = full_names
    list_short_names[f'variable_names_{ctry}_{yyr}'] = var_sample


    TT_FINAL = pd.concat([TT_FINAL, TT_results], ignore_index=True)
    
print('FINISHED')




## GENERATE DATA FOR REGRESSION ANALYSIS - TABLE 1

unique_pairs = TT_FINAL[['country', 'year']].drop_duplicates().reset_index(drop=True)


# Count occurrences of each unique entry in 'country'
count_occurrences = TT_FINAL['country'].value_counts()

# Initialize an empty DataFrame for regression data
TT_regression = pd.DataFrame()

# Iterate over each unique country with more than 10 occurrences
for country in count_occurrences[count_occurrences > 10].index:
    
    # Filter TT_FINAL for the current country
    TT_int = TT_FINAL[TT_FINAL['country'] == country].copy().reset_index(drop=True)
    
    # Convert 'year' to numeric and get unique years
    years = pd.to_numeric(TT_int['year'], errors='coerce')
    unique_years = years.unique()
    
    # Find the minimum and maximum years
    min_year = TT_int['year'].min()
    max_year = TT_int['year'].max()
    
    # Find all indices where 'year' is equal to the min_year and max_year
    z_initial = TT_int[TT_int['year'] == min_year].index
    z_final = TT_int[TT_int['year'] == max_year].index
    
    
    names_initial = TT_int.loc[z_initial, 'var_sample']
    names_final = TT_int.loc[z_final, 'var_sample']

    # Finding the intersection of 'var_sample' names between initial and final
    vars = set(names_initial) & set(names_final)

    # Define empty lists to store the indices
    zz_initial = []
    zz_final = []

    # Iterate over each variable name in 'vars' to find its occurrences within the initial and final years
    for var in vars:
        # Filter TT_int for rows where 'var_sample' matches the current variable name
        matching_rows = TT_int[TT_int['var_sample'] == var].index

        # Intersect matching_rows with z_initial and z_final using pandas Index intersection
        intersection_initial = matching_rows.intersection(z_initial)
        intersection_final = matching_rows.intersection(z_final)

        # Store the first index of the intersection if it exists; otherwise, store NaN
        zz_initial.append(intersection_initial[0] if not intersection_initial.empty else np.nan)
        zz_final.append(intersection_final[0] if not intersection_final.empty else np.nan)

    # Convert lists to pandas Index for further consistency in indexing if needed
    zz_initial = pd.Index(zz_initial).dropna().astype(int)
    zz_final = pd.Index(zz_final).dropna().astype(int)
            
    # Extract the initial country-year pair
    initial_pair = TT_int.iloc[z_initial][['country', 'year']]

    # Assuming 'unique_pairs' is a DataFrame with a reset index
    # Find the row in 'unique_pairs' that matches 'initial_pair'
    phi_loc = unique_pairs.reset_index().merge(initial_pair, on=['country', 'year'], how='inner')['index'].values[0]

    # Using the found index (phi_loc) to access the relevant data
    PHI_initial_key = f'phi_{country}_{min_year}'
    short_var_name_key = f'variable_names_{country}_{min_year}'

    PHI_initial = PHI_NORMALIZED.get(PHI_initial_key, None)
    short_var_names = list_short_names.get(short_var_name_key, [])

    # Filter PHI_initial based on the intersection of vars and short_var_names
    if PHI_initial is not None:
        # Find indices of vars in short_var_names
        indices = [short_var_names.index(var) for var in vars if var in short_var_names]

        # Filter PHI_initial using numpy's advanced indexing
        PHI_initial_filtered = PHI_initial[np.ix_(indices, indices)]
        
        
    # Calculate changes for each metric between initial and final observations
    change_MPI = TT_int.loc[zz_final, 'MPI'].values - TT_int.loc[zz_initial, 'MPI'].values
    change_A = TT_int.loc[zz_final, 'A_int'].values - TT_int.loc[zz_initial, 'A_int'].values
    centrality_w_norm_final = TT_int.loc[zz_final, 'centrality_w_norm'].values
    # Assuming change_A is a single value or a numpy array that matches the dimensions for multiplication with PHI_initial
    # If PHI_initial is a DataFrame, ensure it's converted to a numpy array for this operation, if not already compatible
    
    # Calculate the number of years between the initial and final observations
    # Assuming unique_yyrs is a numpy array or a pandas Series of unique years present in TT_int
    change_years = max(unique_years) - min(unique_years)

    # Normalize the changes by the number of years
    # These operations assume change_years is a scalar. If change_years is an array, adjustments might be needed
    change_A /= change_years
    
    
    # Prepare the DataFrame for regression analysis by selecting the initial observation
    TT_int_for_regression = TT_int.loc[zz_initial, :].copy()

    # Add calculated changes to the DataFrame
    TT_int_for_regression['change_MPI'] = change_MPI
    TT_int_for_regression['change_A'] = change_A
    TT_int_for_regression['change_years'] = change_years
    TT_int_for_regression['centrality_w_norm_final'] = centrality_w_norm_final

    # Append the prepared DataFrame to the TT_regression DataFrame
    TT_regression = pd.concat([TT_regression, TT_int_for_regression], ignore_index=True)
    

## ADD INSTRUMENTAL


unique_regression_pairs = TT_regression[['country', 'year']].drop_duplicates().reset_index(drop=True)

# Initialize a matrix to hold the instrumental correlations
CORRELATION_INSTRUMENTAL = np.nan * np.zeros((len(unique_regression_pairs), len(unique_regression_pairs)))

for i in range(len(unique_regression_pairs)):
    for j in range(len(unique_regression_pairs)):
        # Dynamically construct keys based on country and year
        key_i = f"phi_{unique_regression_pairs.iloc[i]['country']}_{unique_regression_pairs.iloc[i]['year']}"
        key_j = f"phi_{unique_regression_pairs.iloc[j]['country']}_{unique_regression_pairs.iloc[j]['year']}"
        names_key_i = f"variable_names_{unique_regression_pairs.iloc[i]['country']}_{unique_regression_pairs.iloc[i]['year']}"
        names_key_j = f"variable_names_{unique_regression_pairs.iloc[j]['country']}_{unique_regression_pairs.iloc[j]['year']}"

        
        PHI_i = PHI_NORMALIZED[key_i]
        names_i = list_short_names[names_key_i]

        PHI_j = PHI_NORMALIZED[key_j]
        names_j = list_short_names[names_key_j]

        # Find common variables
        vars = list(set(names_i) & set(names_j))
        
        # Indices of common variables in PHI matrices
        zz_1 = [names_i.index(var) for var in vars]
        zz_2 = [names_j.index(var) for var in vars]
        
        # Selecting the relevant rows and columns from PHI matrices based on common variables
        PHI_i_reduced = PHI_i[np.ix_(zz_1, zz_1)]
        PHI_j_reduced = PHI_j[np.ix_(zz_2, zz_2)]
        
        # Calculate correlation between flattened matrices
        corr_value = np.corrcoef(PHI_i_reduced.flatten(), PHI_j_reduced.flatten())[0, 1]
        CORRELATION_INSTRUMENTAL[i, j] = corr_value

  #  print(f"Processed pair {i+1} of {len(unique_regression_pairs)}")
        
# Adjust the correlation matrix
CORRELATION_INSTRUMENTAL -= np.eye(len(unique_regression_pairs))

# Find max correlation indices
max_indices = np.argmax(CORRELATION_INSTRUMENTAL, axis=1)

# Initialize centrality_w_norm_inst as NaNs
centrality_w_norm_inst = np.nan * np.ones(len(TT_regression['centrality_w_norm']))

for i in range(len(unique_regression_pairs)):
    country_i, year_i = unique_regression_pairs.iloc[i]['country'], unique_regression_pairs.iloc[i]['year']
    country_j, year_j = unique_regression_pairs.iloc[max_indices[i]]['country'], unique_regression_pairs.iloc[max_indices[i]]['year']

    # Filter TT_regression_2 for the current and max correlated pairs
    pair_i = TT_regression[(TT_regression['country'] == country_i) & (TT_regression['year'] == year_i)]
    pair_j = TT_regression[(TT_regression['country'] == country_j) & (TT_regression['year'] == year_j)]

    # Common variables
    vars = list(set(pair_i['var_sample']) & set(pair_j['var_sample']))

    # Find indices in TT_regression_2 for vars in both pairs
    for var in vars:
        index_i = pair_i[pair_i['var_sample'] == var].index
        index_j = pair_j[pair_j['var_sample'] == var].index

        # Update centrality_w_norm_inst based on common variables
        centrality_w_norm_inst[index_i] = pair_j.loc[index_j, 'centrality_w_norm'].values
    
TT_regression['centrality_w_norm_inst'] = centrality_w_norm_inst

## GENERATE DATA FOR 3 PERIOD REGRESSION

# Get unique pairs from the first two columns
unique_pairs = TT_FINAL.iloc[:, 0:2].drop_duplicates()

# Convert country column to string type, find unique entries and their counts
unique_entries, entry_counts = np.unique(TT_FINAL['country'].astype(str), return_counts=True)
count_occurrences = np.bincount(entry_counts)

TT_regression_3_period = pd.DataFrame()
for i, unique_entry in enumerate(unique_entries):
    if (unique_pairs['country'] == unique_entry).sum() == 3:
        indices = TT_FINAL[TT_FINAL['country'] == unique_entry].index
        TT_int = TT_FINAL.loc[indices]

        # Convert 'year' to numeric and get unique years
        years = pd.to_numeric(TT_int['year'], errors='coerce')
        unique_years = np.sort(years.unique())
    

        for KK in range(len(unique_years) - 1):
            indices_initial = TT_int[years == unique_years[KK]].index
            indices_final = TT_int[years == unique_years[KK + 1]].index

            names_initial = TT_int.loc[indices_initial, 'var_sample']
            names_final = TT_int.loc[indices_final, 'var_sample']

            vars = np.intersect1d(names_initial, names_final)
            zz_initial = np.full(vars.shape, np.nan)
            zz_final = np.full(vars.shape, np.nan)

            for j, var in enumerate(vars):
                z = TT_int[TT_int['var_sample'] == var].index
                zz_initial[j] = np.intersect1d(indices_initial, z)
                zz_final[j] = np.intersect1d(indices_final, z)

            initial_pair = TT_int.loc[indices_initial, :].iloc[:, 0:2].drop_duplicates()
            
             # Using the found index (phi_loc) to access the relevant data
            PHI_initial_key = f'phi_{country}_{unique_years[KK]}'
            short_var_name_key = f'variable_names_{country}_{unique_years[KK]}'

            PHI_initial = PHI_NORMALIZED.get(PHI_initial_key, None)
            short_var_names = list_short_names.get(short_var_name_key, [])

            # Filter PHI_initial based on the intersection of vars and short_var_names
            if PHI_initial is not None:
                # Find indices of vars in short_var_names
                indices = [short_var_names.index(var) for var in vars if var in short_var_names]

                # Filter PHI_initial using numpy's advanced indexing
                PHI_initial_filtered = PHI_initial[np.ix_(indices, indices)]
                
                    # Calculate changes for each metric between initial and final observations
            change_MPI = TT_int.loc[zz_final, 'MPI'].values - TT_int.loc[zz_initial, 'MPI'].values
            change_A = TT_int.loc[zz_final, 'A_int'].values - TT_int.loc[zz_initial, 'A_int'].values


            # Calculate the number of years between the initial and final observations
            # Assuming unique_yyrs is a numpy array or a pandas Series of unique years present in TT_int
            change_years = unique_years[KK+1] - unique_years[KK]
            

            # Normalize the changes by the number of years
            # These operations assume change_years is a scalar. If change_years is an array, adjustments might be needed
            change_A /= change_years


            # Prepare the DataFrame for regression analysis by selecting the initial observation
            TT_int_for_regression = TT_int.loc[zz_initial, :].copy()

            # Add calculated changes to the DataFrame
            TT_int_for_regression['change_MPI'] = change_MPI
            TT_int_for_regression['change_A'] = change_A
            TT_int_for_regression['change_years'] = change_years
            TT_int_for_regression['period'] = KK            

            # Append the prepared DataFrame to the TT_regression DataFrame
            TT_regression_3_period = pd.concat([TT_regression_3_period, TT_int_for_regression], ignore_index=True)
            
            
print('FINISHED')            

# Figure 2A
correlation_coef = np.corrcoef(TT_FINAL['centrality_w_norm'], TT_FINAL['A_int'])[0, 1]
slope, intercept, r_value, p_value, std_err = linregress(TT_FINAL['centrality_w_norm'], TT_FINAL['A_int'])
regression_line = slope * TT_FINAL['centrality_w_norm'] + intercept

fig = make_subplots(rows=2, cols=3, subplot_titles=('a', 'b', 'c', 'd'), horizontal_spacing=0.08, vertical_spacing=0.15,
                    specs=[[{"type": "scatter"}, {"type": "bar"}, {"type": "box"}],
                           [{"colspan": 3}, None, None]])

# Scatter plot
fig.add_trace(go.Scatter(
    x=TT_FINAL['centrality_w_norm'],
    y=TT_FINAL['A_int'],
    mode='markers',
    marker=dict(color='rgba(31, 119, 180, 0.8)', size=8, line=dict(color='DarkSlateGrey', width=1)),
    name='Data Points'
), row=1, col=1)

# Regression line
fig.add_trace(go.Scatter(
    x=TT_FINAL['centrality_w_norm'],
    y=regression_line,
    mode='lines',
    line=dict(color='rgba(255, 0, 0, 0.8)', width=2, dash='dash'),
    name='Regression Line'
), row=1, col=1)

fig.update_xaxes(title_text='Poverty Centrality', showline=True, linewidth=2, linecolor='black', row=1, col=1,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))
fig.update_yaxes(title_text='Censored Headcount Ratio', showline=True, linewidth=2, linecolor='black', row=1, col=1,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))

fig.add_annotation(
    x=0.01, y=0.98, xref='paper', yref='paper',
    text=f'Correlation Coefficient: {correlation_coef:.2f}',
    showarrow=False,
    font=dict(size=12, color='black'),
    bgcolor='white',
    opacity=1
)

# Figure 2B
median_indicator = TT_regression.groupby('full_names')['centrality_w_norm_final'].median().reset_index()
median_indicator_sorted_general = median_indicator.sort_values(by='centrality_w_norm_final')

# Horizontal bar plot
fig.add_trace(go.Bar(
    x=median_indicator_sorted_general['centrality_w_norm_final'],
    y=median_indicator_sorted_general['full_names'],
    orientation='h',
    marker=dict(color='rgba(31, 119, 180, 0.8)', line=dict(color='DarkSlateGrey', width=1))
), row=1, col=2)

fig.update_xaxes(title_text='Median Poverty Centrality', showline=True, linewidth=2, linecolor='black', row=1, col=2,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True, row=1, col=2,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))

# Figure 2C
sorted_full_variable_names = median_indicator_sorted_general['full_names'].tolist()
sorted_indices = [full_variable_names.index(name) for name in sorted_full_variable_names]

for i in range(len(sorted_indices)):
    column_data = Correlation_matrix[:, sorted_indices[i]]
    fig.add_trace(go.Box(
        x=column_data[~np.isnan(column_data)],
        name=sorted_full_variable_names[i],
        orientation='h',
        boxmean='sd',
        marker_color='rgba(31, 119, 180, 0.8)'
    ), row=1, col=3)

fig.update_layout(
    width=1200,
    height=1000,
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=False
)

fig.update_xaxes(title_text='Correlation Coefficient', showline=True, linewidth=2, linecolor='black', row=1, col=3,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))
fig.update_yaxes(showticklabels=False, row=1, col=3)  # Hide y-axis labels for 2C

# Align y-axis labels for 2B and 2C
fig.update_yaxes(tickmode='array', tickvals=list(range(len(sorted_full_variable_names))), ticktext=sorted_full_variable_names, row=1, col=2,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))

# Figure 2D
unique_countries = TT_regression['country'].unique()

# Initialize a list to store correlation values for each country
corr_indicators = []

# Calculate the correlation for 'centrality_w_norm' and 'centrality_w_norm_final' for each country
for country in unique_countries:
    # Filter the DataFrame for the current country
    filtered_df = TT_regression[TT_regression['country'] == country]
    
    # Calculate correlation and append to list
    correlation = filtered_df['centrality_w_norm'].corr(filtered_df['centrality_w_norm_final'])
    corr_indicators.append(correlation)

# Convert the list of correlations to a NumPy array for sorting
corr_indicators = np.array(corr_indicators)

# Sort the correlations and get the sorted indices
sorted_indices = np.argsort(corr_indicators)

# Vertical bar plot for correlations
fig.add_trace(go.Bar(
    x=np.array(unique_countries)[sorted_indices],
    y=corr_indicators[sorted_indices],
    marker=dict(color='rgba(31, 119, 180, 0.8)', line=dict(color='DarkSlateGrey', width=1))
), row=2, col=1)

fig.update_xaxes(title_text='Country', showline=True, linewidth=2, linecolor='black', row=2, col=1,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))
fig.update_yaxes(title_text='Poverty centrality correlation in initial and final years of survey', showline=True, linewidth=2, linecolor='black', row=2, col=1,
                 titlefont=dict(color='black'), tickfont=dict(color='black'))

fig.update_layout(height=800)

fig.show()

# Export the figure as a high-quality PDF
pio.write_image(fig, 'output/figureA21.pdf', format='pdf', width=1200, height=800, scale=2)